In [1]:
# each row in the result produced by these codes is a tableau, for example [[1,2,4],[3,5,6]]. Here 124, 356 are the two columns of the tableau.


'''Generating grassmannians tableaux'''
#Import libraries
import numpy as np
import sympy as sp
from typing import List
from sage.combinat.shuffle import ShuffleProduct
import itertools
from numpy import matrix 
from itertools import combinations as comb
import multiprocessing 
#from sage.parallel.multiprocessing_sage import Pool
from operator import add
import operator
from sys import getsizeof
import random
from numpy import array
from collections import Counter
import functools
import copy
import scipy 
import matplotlib.pyplot as plt
import networkx as nx
from multiprocessing import get_context
from sage.combinat.root_system.associahedron import Associahedra
from fractions import Fraction
from scipy.special import bernoulli, zeta 

 
####################################################################
#Define relevant functions

def matrixMutation_rectangular(mat,  k):  # mutate at k
    a,b=mat.nrows(), mat.ncols()
    r=Matrix(a,b)
    for i in range(a):
        for j in range(b):
            r[i,j]=mat[i,j]
    
    for i in range(a): 
        for j in range(b): 
            
            if k==i or k==j:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j]+1/2*(abs(mat[i,k])*mat[k,j]+mat[i,k]*abs(mat[k,j]))
     
    return r

def initial_cluster_variables_CDFL(k,n): # Grkn
    t1=InitialClusterWithoutNumericalMatrices(k,n)[1][0]
    t2=TakeIthElements(2,t1)
    r=t2
    return r

def MinorsToPluckerPolynomial(R,M,l): 
    #R is a polynomial ring
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #for i in range(k):
    #    for j in range(n):
    #        var('y'+'_'+str(i)+'_'+str(j))
    #M=Matrix(R, k, n, R.gens())
    #l is a polynomial in y_{i,j}
    # change l = yij ykl + ... to polynomial of Pluckers [[a, [1,2,3],[2,5,8],[4,5,9]], ...]

    p1=l.expand()
    #print(p1)
    #p1=l
    p2=str(p1).split('+')
    #print(p2)
    r=[]
    j=1
    while p1!=0 and j<=len(p2):
        t3=p2[-j]
        t4=change_polynomial_to_list(R, t3)
        sn=0
        for e1 in t4:
            t2=MinorsToPlucker(R,M,e1)  # get a tableau with coefficient
            if IsSemistandardTableauInMatrix(TableauToMatrix(t2[1]))==1:
                r.append(t2)
                #print(t2)
                t1=PluckerToMinors(M, t2)
                p1=(p1-t1).expand()
                p2=str(p1).split('+')
                j=1
                sn=1
                break
        if sn==0:
            j=j+1
            
    return r 

def sub_matrix_general2(R, M, c1, c2):
    m,n=len(c1),len(c2)
    r=Matrix(R, m, n)
    for i in range(m):
        for j in range(n):
            #print(i,j,c1[i],c2[j])
            r[i,j]=M[c1[i]-1, c2[j]-1]
    return r

def matrixMutation_C_matrix(mat, mat_B, k):  # mutate at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if j==k:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j] - max( -mat[i,k], 0 )*mat_B[k,j] + mat[i,k]*max(-mat_B[k,j], 0)
     
    return r

def SubtractVlueOfTwoLists(a,b):
    b1=[-i for i in b]
    r=AddVlueOfTwoLists(a,b1)
    return r

def ScalarMultiplicationForVector(k, a):
    r=[]
    for i in a:
        r.append(k*i)
    return r

def mutation_g_vector_DFGK(kk, g, mat_B, mat_C, mat_B0): # g is the list of initial g vectors, mutate at vertex kk (the vertices are labelled by 0, 1, 2, ...), mat_B0 is the initial exchange matrix

    r=copy.deepcopy(g)
    n=len(g)
    t1=[]
    for i in range(n):
        t1.append(0)
    for i in [0..n-1]:
        #print('i,kk',i,kk,max(-mat_B[i,kk], 0), g[i])
        t1=AddVlueOfTwoLists(t1, ScalarMultiplicationForVector(max(-mat_B[i,kk], 0), g[i]))
    t2=[]
    for i in range(n):
        t2.append(0)
        
    for j in [0..n-1]:
        bj = []
        for i in range(n):
            bj.append(mat_B0[i,j])
        #print('j,kk',j,kk,max(mat_C[j,kk], 0), bj)
        t2=AddVlueOfTwoLists(t2, ScalarMultiplicationForVector(max(mat_C[j,kk], 0), bj))
        
    
    r1=ScalarMultiplicationForVector(-1, g[kk])
    r1=AddVlueOfTwoLists(r1, t1)
    r1=AddVlueOfTwoLists(r1, t2)
    
    r[kk]=r1
    
    return r

def compute_highest_weight_tableau(a1, k,n): # a1=[[1,[[2,3,4,6],[3,4,5,7]]], ...]
    r=[]
    if a1!=[]:
        r0=a1[0][1]
        e1=TableauxWithSameContentAsT_new(r0)
        p1=[]
        for i in [1..len(e1)]:
            M1=randomMatrix(k,n,2,23)
            g1=PluckerToMinorsPolynomial(M1, a1) 
            p1.append([M1, g1])
        e2=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        g1=HighestWeightTableauInPolynomialOfTableaux(e2, 'A', k-1)
        g2=RemoveTrivialTableau(SortRowsOfTableau(g1[1]), k,n)
        r=[g2, e2]
    return r

def TakeExponentsOfMonomial(R,m, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r1=change_polynomial_to_list(R,m)
    r2=r1[0]
    r=[]
    for i in l:
        t1=0
        for j in r2:
            if i==j:
                t1=t1+1
        r.append(t1)
    return r

def TakeExponentsOfMonomial_list(m, l): # m = [2, x11, x12]  l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=m
    r=[]
    for i in l:
        t1=0
        for j in r2:
            if i==j:
                t1=t1+1
        r.append(t1)
    return r

def TakeExponentsOfPolynomial(R,p, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r=[]
    p1=change_polynomial_to_list(R,p)
    for i in p1:
        r.append(TakeExponentsOfMonomial_list(i,l))
    
    return r

def TakeExponentsOfMonomialLaurent(R,m, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r1=m.numerator()
    r2=m.denominator()
    t1=TakeExponentsOfMonomial(R,r1,l)
    t2=TakeExponentsOfMonomial(R,r2,l)
    r=[]
    for i in range(len(t1)):
        r.append(t1[i]-t2[i])
    
    return r

def TakeExponentsOfPolynomialLaurent(R,p, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r=[]
    p1=TakeExponentsOfPolynomial(R,p.numerator(),l)
    p2=TakeExponentsOfPolynomial(R,p.denominator(),l)[0]
    for i in p1:
        t1=[]
        for j in range(len(i)):
            t1.append(i[j]-p2[j])
        r.append(t1)
    
    return r

def compute_inverse_map_of_rational_functions(l,k, n): # l is a list of rational function in xij
    Xs = Matrix([[var("x_%s_%s"%(u, v), latex_name="x_{%s,%s}"%(u, v))
              for v in range(1, k)]
             for u in range(1, n-k)])
    Ys = Matrix([[var("y_%s_%s"%(u, v), latex_name="y_{%s,%s}"%(u, v))
                  for v in range(1, k)]
                 for u in range(1, n-k)])
    #print(Xs, Ys)
    va=Xs.transpose().list()  # if use mormal order, Xs.list()
    a1=Ys.transpose().list()  
    
    #print('l', l)
    #print('va', va)
    
    a3 = list(map(lambda a, b:a-b, l, a1))
    
    #print('a3',a3)
    
    Sol = solve(a3, va, solution_dict=True)
    
    #print(Sol)
    for i in Sol:
        t1=list(i.values())
        if IsInList(0, t1)==0:
            a5=t1
            break
    r=a5
    return r

def evaluation_tropical_function_rational_list_on_vector_inverse(R, x, k,n,   f0, vv):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f0 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    
    f1=compute_inverse_map_of_rational_functions(f0,k, n)           # compute inverse map of f0
    
    indices = [(i,j) for i in range(k) for j in range(n)]
    R1 = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    y = dict(zip(indices, R1.gens()))
    
    for i in f1:
        t1=evaluation_tropical_function_rational_on_vector(R1, y, k,n,  i, vv)
        r.append(t1)

    return r

def evaluation_tropical_function_on_vector(R, x, k,n,  f1, vv): 
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=0
    value_x={}
    
    for i in range(k-1):
        for j in range(n-k-1):
            value_x[repr(x[j+1,i+1])]=vv[i*(n-k-1)+j]
    #print(value_x)
    r1=[]
    for i in f1:
        t1=0
        for j in [1..len(i)-1]:
            t1=t1+value_x[repr(i[j])]
        r1.append(t1)
    #print(r1)
    r=min(r1)
    return r

def evaluation_tropical_function_rational_on_vector(R, x, k,n,  f1, vv):  # f1 is a rational function
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=f1.numerator()
    r3=f1.denominator()
    c1=change_polynomial_to_list(R,r2)
    c2=change_polynomial_to_list(R,r3)
    
    g1=evaluation_tropical_function_on_vector(R, x, k,n,   c1, vv)
    g2=evaluation_tropical_function_on_vector(R, x, k,n,   c2, vv)
    r=g1-g2
    
    #print(f1)
    #print(c1)
    #print(c2)
    #print(g1,g2,r)

    return r

def evaluation_tropical_function_rational_list_on_vector(R, x, k,n,   f1, vv):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f1 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    for i in f1:
        t1=evaluation_tropical_function_rational_on_vector(R, x, k,n,  i, vv)
        r.append(t1)

    return r

def ModuleToTableauSLk(Y,l,k): # SLk
    t1=ModuleToMultiSegment(Y,l)
    r=MultisegmentToTableauSLk(k,t1)
    return r

def MultisegmentToTableauSLkOneSegment(k, m): # SLk
 
    a=m[0]
    b=m[1]
    t1=ListAToN(1, -a)
    t1.append(k-b)
    r=[t1]

    return r


def MultisegmentToTableauSLk(k, l): # SLk

    r=[]
    if l!=[]:
        for i in l:
            t1=MultisegmentToTableauSLkOneSegment(k, i)
            #print('t1', t1)
            r=UnionOfTwoTableauxSLk(r, t1)
            
        #print('r', r)
             
        r=RemoveTrivialTableauSLk(r,k) 
        #print(r)
    return r

def PluckersWhichAreOneSLk(k): # SLk 1, 12, 123, ..., are trivial
    r=[]
    for i in [1..k]:
        t1=[]
        for j in [1..i]:
            t1.append(j)
        r.append(t1)
    return r

def IsTableauAFactorOfBSLk(a,b):
    
    r1=a
    r2=b

    r=1;
    for i in range(len(r1)):
        if IsSubsetAsMultiset(r1[i], r2[i])==0:
            r=0
            break
    return r

def RemoveTrivialTableauSLk(a1,k): # SLk
    r=a1
    t2=PluckersWhichAreOneSLk(k);
    sn=1
    while sn!=0:
        r1=r;
        for i in range(len(t2)):
            #print(r, t2[i])
            if r==[]:
                break
            elif IsTableauAFactorOfBSLk([t2[i]], r)==1:
                p1=TableauDivisionSLkResultInRows(r, [t2[i]])
                #print('p1',p1)
                if IsSemistandardTableauInRows(p1)==1:
                    r=TableauDivisionSLk(r, [t2[i]])
                
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1
        
    
    return r

def TableauToModuleSLk(Y,k,l): # l is a tableau T, l[i] is column of i of T, SLk
    
    t1=TableauToFundamentalTableauxSLk(Y,l)
    #print(t1)
 
    r=[]
    for i in t1:
        r.append(FundamentalTableauToYiaSLk(i,k)) 

    return r

def FundamentalTableauToYiaSLk(a1,k): # SLk, T_{l,a}, length l, last entry a, T_{l,a}=T^{l+k-a,l} \sim Y_{l+k-a,k-a-l}, weight of T_{l,a} is omiga_(l+k-a)
 
    r=[]
    rank=k-1
    n=rank
    for i in [1..n]:
        r=append(r,0)
        
    a=sorted(a1)
    l=len(a)
    t1=a[len(a)-1]
    r=[l+k-t1,k-t1-l]
    
    return r

def IsPartition1GreaterThanOrEqualPartition2(p1,p2): # p1,p2 are partitions
    l1=p1
    l2=p2

    n=max(len(p1), len(p2))
    for i in [len(p1)..n]:
        l1.append(0) 
    for i in [len(p2)..n]: 
        l2.append(0) 

    r=1 # p1 >= p2

    for i in [1..n]: 
        t1=0
        t2=0
        for j in [0..i-1]:
            t1=t1+l1[j]
            t2=t2+l2[j] 
        if t1<t2: 
            r=0 # p1 is not greater or equal to p2
            break 
    return r

def comparePartitions(p1,p2): # p1,p2 are partitions
    t1=IsPartition1GreaterThanOrEqualPartition2(p1,p2)
    t2=IsPartition1GreaterThanOrEqualPartition2(p2,p1)

    if t1==1: 
        r=1  # p1>=p2
    elif t2==1: 
        r=-1 # p1<=p2
    else:
        r=0 # not comparable 
    return r

def IsTableauxP1GreaterEqualP2SLkU(P1,P2): # P1,P2 are tableaux

    n=-1000
    for i in UnionOfListsMany(P1):
        if n<i:
            n=i
    for i in UnionOfListsMany(P2):
        if n<i:
            n=i 

    r=1 # P1>=P2

    for a in [1..n]: 
        t1=TableauRestrictToElementsLessOrEqualAToPartition(P1,a)
        t2=TableauRestrictToElementsLessOrEqualAToPartition(P2,a)
        r1=comparePartitions(t1,t2)
        if r1==0 or r1==-1: 
            r=0 # P1 is not greater or equal to P2
            break 

    return r

def TableauToMatrixTakeRowsSLkMoreGeneral(a): # send a tableau to a matrix, a[i] are rows
    if a==[]:
        r=[]
    else:
        m=len(a)  # number of rows
        n=len(a[0])
        for i in [1..len(a)-1]:
            if n<len(a[i]):
                n=len(a[i])
        
        r=Matrix(m,n)
        for i in range(m):
            for j in range(n):
                if j<len(a[i]):
                    r[i,j]=a[i][j]
                else:
                    r[i,j]=-10000
    return r

def TableauToMatrixTakeColumnsSLkMoreGeneral(a): # send a tableau to a matrix, a[i] are columns
    
    if a==[]:
        r=[]
    else:
        n=len(a)  # number of columns
        m=len(a[0])
        for i in [1..len(a)-1]:
            if m<len(a[i]):
                m=len(a[i])
        
        r=Matrix(m,n)
        for i in range(m):
            for j in range(n):
                if i<len(a[j]):
                    r[i,j]=a[j][i]
                else:
                    r[i,j]=-10000
    return r 

def TableauToMatrixTakeRowsSLkListMoreGeneral(a): # SLk, a[i] are columns of tableau
    r=[]
    for i in a:
        r.append(TableauToMatrixTakeRowsSLkMoreGeneral(i))
    return r

def TableauToMatrixTakeColumnsSLkListMoreGeneral(a): # SLk, a[i] are columns of tableau
    r=[]
    for i in a:
        r.append(TableauToMatrixTakeColumnsSLkMoreGeneral(i))
    return r

def TableauToPartition(P):

    r=[]
    t1=TableauToMatrixTakeColumnsSLkMoreGeneral(P)
    for i in [0..t1.nrows()-1]: 
        t2=0
        for j in [0..t1.ncols()-1]: 
            if t1[i,j]!=-10000:
                t2=t2+1 
        r.append(t2) 
    return r

def TableauRestrictToElementsLessOrEqualAToPartition(P,a):

    t1=TableauRestrictToElementsLessOrEqualA(P,a)
    r=TableauToPartition(t1)

    return r

def compareWeightsTableauxSLkU(P1,P2): # P1,P2 are tableaux

    t1=IsTableauxP1GreaterEqualP2SLkU(P1,P2)
    t2=IsTableauxP1GreaterEqualP2SLkU(P2,P1)

    if t1==1:  
        r=1 # P1>=P2
    elif t2==1: 
        r=-1 # P1<=P2
    else:
        r=0 # P1, P2 not comparable
     
    return r

def TableauRestrictToElementsLessOrEqualA(P,a):

    r=[]
    for i in P: 
        t1=[]
        for j in i: 
            if j<=a:
                t1.append(j) 
        r.append(t1) 
    return r

# mutation for base affine space, C[SL_k]^U, and C[N]

def InitialCluster_SLk(kk): # initial cluster for SL_kk, only works for kk <= 10

    # quivers are from Keller's mutation app

    if kk==2: 
        mat=Matrix([[0,-1,1],[1,0,-1],[-1,1,0]]) # SL2
    elif kk==3: 
        mat=Matrix([[0,-1,1,0,0,0],[1,0,-1,-1,1,0],[-1,1,0,0,-1,1],[0,1,0,0,-1,0],[0,-1,1,1,0,-1],[0,0,-1,0,1,0]]) # SL3
    elif kk==4:
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0],[0,-1,1,1,0,-1,0,-1,1,0],[0,0,-1,0,1,0,0,0,-1,1],[0,0,0,1,0,0,0,-1,0,0],[0,0,0,-1,1,0,1,0,-1,0],[0,0,0,0,-1,1,0,1,0,-1],[0,0,0,0,0,-1,0,0,1,0]]) # SL4
    elif kk==5: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0]]) # SL5
    elif kk==6: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0]]) # SL6
    elif kk==7: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0]]) # SL7
    elif kk==8: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0]]) # SL8
    elif kk==9: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0]]) # SL9
    elif kk==10: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0]]) # SL10

    vertices0=[]
    n=kk

    for i in [1..n]: 
        for j in [1..i]: 
            t3=ListAToN(j, j+n-i)
            vertices0.append(t3) 
            
    verticesTableaux=[] # Tableaux are represented by matrices
    for i in [0..len(vertices0)-1]:
        verticesTableaux.append([0, [vertices0[i]], i]) # [vertices0[i]] is an one column tableau
        
    mat1 = mat
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r
    
def UnionOfTwoTableauxSLk(a,b): # a[i], b[i] are columns
    if a==[]:
        r=b
    elif b==[]:
        r=a
    else:
        r1=TransposeTableauSLk(a)
        r2=TransposeTableauSLk(b)
        
        m=max(len(r1), len(r2))
        #print('m,len(r1),len(r2)',m,len(r1),len(r2))
        r3=[]
        for i in [0..m-1]:
            #print(i,len(r1),len(r2))
            if i<len(r1) and i<len(r2):
                t2=r1[i]+r2[i]
                #print('t2',t2)
                t2=sorted(t2)
            elif i>=len(r1) and i<len(r2):
                    t2=sorted(r2[i])
            elif i<len(r1) and i>=len(r2):
                    t2=sorted(r1[i])
            r3.append(t2)
        #print('r3',r3)
        r=TransposeTableauSLk(r3)
    return r

def PowerOfTableaux_SLk(T, n): # T=[[1,2,3],[2,5]]
    if n==1:
        r=T
    elif n>1:
        r=T
        for i in range(n-1):
            r=UnionOfTwoTableauxSLk(r, T)
    return r
    
def computeEquationsForModulesTableaux_SLk(variable2, mat, k, kk): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableauxSLk(newVariable, PowerOfTableaux_SLk(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableauxSLk( newVariable2, PowerOfTableaux_SLk(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableauxSLkU(newVariable, newVariable2)
    
    if t1==1: 
        variable[k][1]=TableauDivisionSLk(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivisionSLk(newVariable2, variable[k][1])  
    
    #print(variable[k][1])
    clusterVariables=TableauToMatrixTakeColumnsSLkMoreGeneral(variable[k][1])
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def MatrixToTableauTakeColumnsSLk(a): 
 

    n=a.nrows()
    m=a.ncols()

    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            if a[j,i]>=0:
                t2=append(t2,a[j,i])
        
        r=append(r, t2)
    
    #r=SemistandardTableau(r)
   
    return r

def ll_perms_SLk(lli, kk, max_column, repeat): # SL_kk
    b1=[]
    IC=InitialCluster_SLk(kk)
    mat1=IC[0]
    vertices1=IC[1]

    ll=list(map(lambda x: x - 1, lli))
    
    #print(IC)
    
    mutationSequence=[]
    for j1 in [1..repeat]: ###why do this?, just to repeat the same sequence of mutations, it will give more cluster variables, without this loop is also ok
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableaux_SLk(vertices1, mat1, mutationSequence[j], kk)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux_SLk(vertices1, mat1, mutationSequence[j], kk)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            #print(vertices1[1])
            b1.append(MatrixToTableauTakeColumnsSLk(vertices1[1]))

    return b1

def TableauDivisionSLk(a,b): # a[i], b[i] are columns
    r1=TableauDivisionSLkResultInRows(a,b)
        
    r=TransposeTableauSLk(r1)
        
    return r

def TableauDivisionSLkResultInRows(a,b): # a[i], b[i] are columns
    r1=TransposeTableauSLk(a)
    r2=TransposeTableauSLk(b)
    
    r3=[]
    m=max(len(r1),len(r2))
    for i in range(m):
        if i<len(r1) and i<len(r2):
            r3.append(sorted(SetDifferenceListDifference(r1[i],r2[i])))
        elif i<len(r1) and i>=len(r2):
            r3.append(r1[i])
        elif i>=len(r1) and i<len(r2):
            r3=[]
            print('tableau not divisible')
            break
        
    r=r3
        
    return r

def sub_matrix_symbolic(R,M,a,b):
    r=Matrix(SR,len(a),len(b))
    for i in range(len(a)):
        for j in range(len(b)):
            r[i,j]=M[a[i]-1, b[j]-1]
    return r


def PluckerToMinors_symbolic(R,A,l):
    r=l[0]
    for i in l[1]:
        r=r*Minor_symbolic(R,A,ListAToN(1, len(i)), i)
        print(r)
    return r

def PluckerToMinorsPolynomial_symbolic(R,A,l):
    r=0
    for i in l:
        r=r+PluckerToMinors_symbolic(R,A,i) 
    return r


def Minor_symbolic(R,M, rows, cols):
    #r=np.linalg.det(subMatrix(M,rows, cols))
    #r=det(subMatrix(M,rows, cols))
    #r=det(sub_matrix_general(M,rows, cols))
    #r=detSelfDefined(subMatrix(M,rows, cols))
    t1=sub_matrix_symbolic(R,M,rows, cols)
    r=det(t1)
    return r

def TableauxWithSameContentAsT_and_smaller_or_equal_weight_new(T, typ,rank): 
    r1=TableauxWithSameContentAsT_new(T)
    r=ChooseTableauxWithSmallOrEqualWeight(T,r1,typ,rank)
    return r

def TableauxWithSameContentAsT_new(T): 
    if T and T[0]:
        r=list(candidates_new(Counter(sorted(flatten(T))), len(T[0])))
        r1=removeDuplicatesListOfLists(r)
        r=[]
        for i in r1:
            t1=SortRowsOfTableau(i)
            r.append(t1)
        r=removeDuplicatesListOfLists(r)
        return r

@functools.lru_cache()  
def candidates_tableau(content, k, candidate=()): 
    if len(content) < k:
        if len(content) == 0 and candidate:
            yield candidate
        else:
            return
        
    for I in itertools.combinations(content, k):
        # tuples in candidate are in non-decreaseing order
        if candidate and I < candidate[-1]:
            continue
            
        # adding I to candidate is semistandard tableau after reordering rows 
        if is_semistandard_after_reordering(candidate, I)==1:
            yield from candidates_tableau(content - Counter(I), k, candidate + (I,))
            
def is_semistandard_after_reordering(l,J): # l is a tuple, l[i] is (2,3,...), J = (1,3,...)
    r1=[]
    for i in l:
        r1.append(list(i))
    r1.append(list(J))
    r1=SortRowsOfTableau(r1)
    r=IsSemistandardTableauInMatrix(TableauToMatrix(r1))
    return r

def TableauxWithSameContentAsT_in_rows(shape,content,frontline=None): # each result is in terms of rows
    if frontline is None:
        frontline = [0] * len(shape)

    if len(content)==0:
        yield [[0]*s for s in shape]
        return

    e = content[0]
    for row,col in enumerate(frontline):
        if col>=shape[row] or (row>0 and frontline[row-1]<=col):
            continue

        frontline[row] += 1

        for t in TableauxWithSameContentAsT_in_rows(shape,content[1:],frontline):
            if row+1==len(shape) or col>=shape[row+1] or e<t[row+1][col]:
                t[row][col] = e
                yield t

        frontline[row] -= 1
    return

def TableauxWithSameContentAsT(T): # each result is in terms of columns, T[i] is column i
    t1=TransposeTableauSLk(T)
    shape=[]
    for i in t1:
        shape.append(len(i))
    #print(shape)
    content=sorted(flatten(T))
    r1=list(TableauxWithSameContentAsT_in_rows(shape,content,frontline=None))
    #print(r1)
    r1=removeDuplicatesListOfLists(r1)
    r=TransposeOfRectangularTableauList(r1)
    return r

def TransposeOfRectangularTableauList(l):
    r=[]
    for i in l:
        t1=TransposeOfRectangularTableau(i)
        r.append(t1)
    return r

def TableauxWithSameContentAsT_and_smaller_or_equal_weight(T,typ,rank): # each result is in terms of columns, T[i] is column i
    r1=TableauxWithSameContentAsT(T)
    r=ChooseTableauxWithSmallOrEqualWeight(T,r1,typ,rank)
    return r

def MinorsToPluckerPolynomial(R,M,l): 
    #R is a polynomial ring
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #for i in range(k):
    #    for j in range(n):
    #        var('y'+'_'+str(i)+'_'+str(j))
    #M=Matrix(R, k, n, R.gens())
    #l is a polynomial in y_{i,j}
    # change l = yij ykl + ... to polynomial of Pluckers [[a, [1,2,3],[2,5,8],[4,5,9]], ...]

    p1=l.expand()
    #print(p1)
    #p1=l
    p2=str(p1).split('+')
    #print(p2)
    r=[]
    j=1
    while p1!=0 and j<=len(p2):
        t3=p2[-j]
        t4=change_polynomial_to_list(R, t3)
        sn=0
        for e1 in t4:
            t2=MinorsToPlucker(R,M,e1)  # get a tableau with coefficient
            if IsSemistandardTableauInMatrix(TableauToMatrix(t2[1]))==1:
                r.append(t2)
                print(t2)
                t1=PluckerToMinors(M, t2)
                p1=(p1-t1).expand()
                p2=str(p1).split('+')
                j=1
                sn=1
                break
        if sn==0:
            j=j+1
            
            
    return r

def MinorsToPlucker(R,M,l):
    #R is a polynomial ring
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #for i in range(k):
    #    for j in range(n):
    #        var('y'+'_'+str(i)+'_'+str(j))
    #M=Matrix(R, k, n, R.gens())
    #l is a monomial in y_{i,j}
    # change l = [1, yij, ykl,...] to Plucker [a, [1,2,3],[2,5,8],[4,5,9]]
    
    r1=l
    r=[r1[0]]
    r2=[]
    for j in r1:
        t1=index_of_yij(j) # take indices of yij
        if t1!=[]:
            r2.append(t1[1]+1)
    #print(r2)
    r3=[]
    num=len(r2)/k
    for i in range(num):
        t1=[]
        for j in range(k):
            t1.append(r2[j*num+i])
        r3.append(t1)
    
    r3=SortRowsOfTableau(r3)
    
    r.append(r3)

    return r

def IsSemistandardTableauInMatrix(a): 
    r1=MatrixTakeRows(a)
    r=IsSemistandardTableauInRows(r1)
        
    return r



def symbolic_matrix(R,k,n):
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    r = Matrix(R, k, n, R.gens())
    return r

def BraidGroupActionSigmaIOnWindow_symbolic(R,i1, k, l, l1): # l=[v1,v2,...,vd], r=[v1,v2,...,v_{i-1},v_{i+1},w_1,v_{i+2},...,v_d], l1= l + ... + beginning of original list 
    i=i1-1
    r=l
    r=removeAnElementInList(i,r)
    r.insert(i+1, 0)
    #print(r)
    
    t1=[]
    for j in [i..i+k]:
        if j!=i+1: 
            t1.append(l1[j])
    
    #print(t1)
    #print(TableauToMatrix_symbolic(R,t1))
    m1=det(TableauToMatrix_symbolic(R,t1))
    
    #print(m1)
    
    t1=[]
    for j in [i+1..i+k]:  
        t1.append(l1[j])  
    m2=det(TableauToMatrix_symbolic(R,t1))
    #print(t1)
    #print(m2)
    w1=[]
    for j in [0..k-1]: 
        w1.append(m1*l[i+1][j]-m2*l[i][j]) 
    #print(w1)
    r[i+1]=w1
    
    return r

def TableauToMatrix_symbolic(R,a): # R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    m=len(a)
    n=len(a[0])
    r=Matrix(R, n, m)
    #print(r)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r
              
def index_of_yij(a): # a = y[i,j]=y_i_j
    r=[int(s) for s in str(a).split('_')[1:]] # take indices of yij
    return r

def BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,i,k,n, M): 

    d=gcd(k,n)
    l=MatrixToTableauTakeColumns(M)
    #print(l)

    r1=[]
    t1=[]

    for j in [0..n-1]:
        if (j+1) % d !=0: 
            t1.append(l[j])
        else:
            t1.append(l[j])
            r1.append(t1)
            t1=[] 
    #print(r1)
    r2=[]
    for j in [0..len(r1)-1]: 
        l1=[]
        for i1 in [j..len(r1)-1]: 
            l1=UnionOfLists(l1, r1[i1]) 
        l1=UnionOfLists(l1, l)
        t1=BraidGroupActionSigmaIOnWindow_symbolic(R,i,k, r1[j], l1)
        r2.append(t1) 
    r=TableauToMatrix_symbolic(R,UnionOfListsMany(r2))

    return r

def change_polynomial_to_list(R, f):
    r=[[c] + sum([[SR(R.gen(i))]*e[i] for i in range(R.ngens()) if e[i] != 0], []) for (e,c) in R(f).dict().items()]
    return r



def ContentOfTableauObtainedFromBraidGroupActionTableau(R, ii, T, k,n, M): # T is a tableau, T[i] is a column
    r=[]
    for pl in T:
        t1=ContentOfTableauObtainedFromBraidGroupActionPlucker(R, ii, pl, k,n, M)
        r=r+t1
    
    return r

def BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, pl, k,n, M): # pl = [1,2,3,8], need to check and correct
    r1=ContentOfTableauObtainedFromBraidGroupActionPlucker(R, ii, pl, k,n, M)
    T=list(split_list(r1, k))
    #u1=NonCrossingTupleWithSameContentAsT_new(T) 
    u1=TableauxWithSameContentAsT_new(T)
    c1=[]
    for j in range(len(u1)):
        M1=randomMatrixNonInt(k,n)
        r1=BraidGroupActionSigmaIOnMatrixRenomalizeColumns(ii,k,n, M1)
        r2=PluckerToMinorsPolynomial(r1, [[1, [pl]]])
        c1.append([M1, r2])
    r=ComputeChTInTermsOfNonCrossingBasis([u1, c1])

    return r

def BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    l=NchooseM(ListAToN(1,n),k)
    for i in l:
        t1=BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r


def MultiplyTwoPolynomialsLists2(a,b):

    r=[]
    for i in a:
        for j in b:
            temp1=i[0]*j[0]
            temp2=sorted(i[1]+j[1])
            r.append([temp1,temp2])
    return r

def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, p, k,n, M, sigma_i_of_plucker_Set): 
    # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    #k,n=4,8
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #ii=1
    #M = Matrix(R, k, n, R.gens())
    #sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M)
    r=[]
    for j in p:
        t1=[[j[0], []]]
        #print('t1',t1)
        for j1 in j[1]:
            #t2=BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, j1, k,n, M)
            t2=sigma_i_of_plucker_Set.get(repr(j1))
            t2=polynomial_use_tuple_to_polynomial_use_list(t2)
            #print(t2)
            t1=MultiplyTwoPolynomialsLists2(t1,t2)
        r=r+t1
    #r=combineLikeTerms3(r)
    return r

def BraidGroupActionSigmaIOnPluckerSet_symbolic_partial(R,ii, k,n, M, l): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    for i in l:
        t1=BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r

def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_NTimes2(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    l=[]
    for i in p:
        l=l+i[1]
    
    sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic_partial(R,ii, k,n, M,l) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r

def define_ring_with_variables_that_have_negative_indices(Y,k,n1,n2): # define ring with variables which have negative indices
    #k,n1,n2=4,-100,2, Y='x'
    indices = [(i,j) for i in range(1,k+1) for j in range(n1, n2)]
    R = PolynomialRing(QQ, [(Y+'_{}_{}').format(i, str(j).replace('-', 'm')) for (i,j) in indices])
    Y = dict(zip(indices, R.gens()))
    r=[R, Y]
    return r

def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_NTimes(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r

def polynomial_use_tuple_to_polynomial_use_list(l):
    r=[]
    for i in l:
        t1=[]
        for j in i[1]:
            t1.append(list(j))
        r.append([i[0], t1])
    return r

def web_matrix(R, x,k, n):
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R1 = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    #x = dict(zip(indices, R.gens()))
    r1=Matrix(SR, k,n)
    for i in range(k):
        r1[i,i]=1
    for i in [k..n-1]:
        r1[k-1,i]=1
    #print(r1)
    for i in [1..k-1]:
        for j in [1..n-k]:
            t1=0
            for p in itertools.combinations_with_replacement(ListAToN(1,j), k-i): # choose with repetition
                t3=1
                for p1 in [0..len(p)-1]:
                    t3=t3*x[i+p1,p[p1]]
                t1=t1+t3
            r1[i-1,k+j-1]=t1
    r=r1
    return r

def split_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def sub_matrix_general(M, c1, c2):
    m,n=len(c1),len(c2)
    r=Matrix(SR, m, n)
    for i in range(m):
        for j in range(n):
            r[i,j]=M[c1[i]-1, c2[j]-1]
    return r

def AddVlueOfListsMany(l):
    r=[]
    if l!=[]:
        n=len(l[0])
        for i in range(n):
            r.append(0)
        for i in l:
            r=AddVlueOfTwoLists(r, i)
    return r

def are_all_non_negative(l):
    r=1
    for i in l:
        if i<0:
            r=0
    return r

def initial_cluster_variables_Yis(typ,rank,l): # mutable variables in the beginning
    r1=InitialCluster_Yis(typ,rank,l)
    r2=r1[2]
    #print(r2, len(r1[1][0]))
    #print(r1[3])
    r3=SetDifference(r1[3],r2)
    #print(r3)
    r=[]
    for i in r2:
        r.append(r1[1][0][i][1])
    for i in r3:
        r.append(r1[1][0][i][1])
    return r

def InitialCluster_Yis(typ, rank, l): # C_l^{Dn}, l \ge 0, at the moment, typ='D'
    if typ=='A':
        n=rank
        p1=n*(l+1)
        
        vertices0=[]

        for i in [1..n]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-2, j+1,typ,rank)
                vertices0.append(sorted(t1))
                
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-2]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        for j in [0..l-1]:
            for i in [0..n-2]:
                t1=(i+1)*(l+1)+j+1
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='B':
        n=rank
        p1=2*n*(l+1)
        
        vertices0=[]

        for i in [1..n-1]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 2*n-2*i, j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in [n]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..2*l+1]:  # j=l are frozens
                t1=KR_module_largest_s(i, 1, j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in range(n-1,0,-1):
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 2*i+2-2*n, j+1,typ,rank)
                vertices0.append(sorted(t1))
                
        #print(vertices0)
        
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
        
        mat=Matrix(p1,p1)
        for i in [0..n-2]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        i=n-1
        for j in [0..2*l]:
            t1=i*(l+1)+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
            
        for i in [n+1..2*n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
                
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l]:
            t1=(n-2)*(l+1)+j
            t2=(n-1)*(l+1)+2*j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            t1=(n-1)*(l+1)+2*j
            t2=(n+1)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for i in [n+1..2*n-2]:
            for j in [0..l]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            t1=(n-1)*(l+1)+2*j+1
            t2=(n-2)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l-1]:
            t1=(n+1)*(l+1)+j
            t2=(n-1)*(l+1)+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [n+1..2*n-2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        
        frozen_vertices=[]
        for i in [0..n-2]:
            frozen_vertices.append(i*(l+1)+l)
        i=n-1
        frozen_vertices.append((n+1)*(l+1)-1)
        for i in [n+1..2*n-1]:
            frozen_vertices.append(i*(l+1)+l)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='C':
        n=rank
        p1=(n-1)*(l+1)+ceil(l/2)+ceil((l-1)/2)+2
        
        vertices0=[]

        for i in [1..n-1]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, -i, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=n
        d=distance_of_indices_KR_module_half(i,typ,rank)
        for j in [0..ceil(l/2)]:  
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=n
        d=distance_of_indices_KR_module_half(i,typ,rank)
        for j in [0..ceil((l-1)/2)]:  
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
                
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-2]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for i in [n-1]:
            for j in [0..ceil(l/2)-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for i in [n]:
            for j in [0..ceil((l-1)/2)-1]:
                t1=(n-1)*(l+1)+ceil(l/2)+1+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..ceil((l-1)/2)]:
            t1=(n-2)*(l+1)+2*j
            t2=(n-1)*(l+1)+ceil(l/2)+1+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..ceil(l/2)-1]:
            t1=(n-2)*(l+1)+2*j+1
            t2=(n-1)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..ceil(l/2)-1]:
            t1=(n-1)*(l+1)+j
            t2=(n-2)*(l+1)+2*j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..ceil((l-1)/2)-1]:
            t1=(n-1)*(l+1)+ceil(l/2)+j+1
            t2=(n-2)*(l+1)+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
  
        frozen_vertices=[]
        for i in [0..n-2]:
            frozen_vertices.append(i*(l+1)+l)
        frozen_vertices=frozen_vertices+[ (n-1)*(l+1)+ceil(l/2), (n-1)*(l+1)+ceil(l/2)+ceil((l-1)/2)+1 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices 
        
        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
    
    elif typ=='D':
        n=rank
        p1=n*(l+1)

        vertices0=[]
        
        for i in [1..n-1]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-2, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=n
        for j in [0..l]:   # j=l are frozens 
            t1=KR_module_largest_s(i, i-3, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j+1
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        i=n-3
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1

        i=n-3
        for j in [0..l-1]:
            t1=(i+2)*(l+1)+j+1
            t2=i*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='E':  # 
        n=rank
        p1=n*(l+1)
        
        vertices0=[]
        xi=[-1,0,1,2]
        num=-1
        for i in [1,3,4,2]:
            num=num+1
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, xi[num], j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in [5..n]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-3, j+1,typ,rank)
                vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=4
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i-2)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            for i in [5..n-1]:
                t1=i*(l+1)+j
                t2=(i-1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..l-1]:
            for i in [0..2]+[4..n-2]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=2
        for j in [0..l-1]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1

        frozen_vertices=[]
        for i in [1..n]:
            frozen_vertices.append(i*l+i-1)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='F':
        n=rank
        m1=ceil(l/2)
        m2=floor(l/2)
        p1=2*l+2*m1+2*m2+7
        
        vertices0=[]
        for i in [4,3]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-5, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=2
        for j in [0..m1]:  # j=l are frozens
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=1
        for j in [0..m1+1]:  # j=l are frozens
            t1=KR_module_largest_s(i, 1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=2
        for j in [0..m2]:  # j=l are frozens
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=1
        for j in [0..m2]:  # j=l are frozens
            t1=KR_module_largest_s(i, -5, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for j in [0..l-1]+[l+1..2*l]+[2*l+2..2*l+m1+1]+[2*l+m1+3..2*l+2*m1+3]+[2*l+2*m1+5..2*l+2*m1+m2+4]+[2*l+2*m1+m2+6..2*l+2*m1+2*m2+5]:
            t1=j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
            
        for j in [0..l]:
            t1=j
            t2=l+1+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2]:
            t1=l+2*j+1
            t2=2*l+2*m1+5+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2]:
            t1=2*l+2*m1+5+j
            t2=2*l+2*m1+m2+6+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [1..m1]:
            t1=l+2*j
            t2=2*l+2+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1]:
            t1=2*l+2+j
            t2=2*l+m1+4+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..l-1]:
            t1=l+1+j
            t2=j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1-1]:
            t1=2*l+2+j
            t2=l+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1]:
            t1=2*l+m1+3+j
            t2=2*l+2+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=2*l+2*m1+5+j
            t2=l+2*j+3
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=2*l+2*m1+m2+6+j
            t2=2*l+2*m1+6+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        
        
        frozen_vertices=[ l, 2*l+1, 2*l+m1+2, 2*l+2*m1+4, 2*l+2*m1+m2+5, 2*l+2*m1+2*m2+6 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='G':
        n=rank
        m1=ceil((l+1)/3)
        m2=ceil(l/3)
        m3=ceil((l-1)/3)
        p1=l+1+m1+m2+m3
        
        vertices0=[]
        i=1
        for j in [0..l]:  # j=l are frozens
            t1=KR_module_largest_s(i, 0, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=2
        for j in [0..m1-1]:   # 
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        for j in [0..m2-1]:   # 
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
        for j in [0..m3-1]:   # 
            t1=KR_module_largest_s(i, -5, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for j in [0..l-1]:
            t1=j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m1-2]:
            t1=l+1+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m2-2]:
            t1=l+1+m1+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m3-2]:
            t1=l+1+m1+m2+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m1-1]:
            t1=j*3
            t2=l+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=j*3+1
            t2=l+m1+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m3-1]:
            t1=j*3+2
            t2=l+m1+m2+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            if j % 3 == 0 and j>=3: 
                t1=l+j/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
            elif j%3 == 1 and j>=4:
                t1=l+m1+(j-1)/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
            elif j%3 == 2 and j>=5:
                t1=l+m1+m2+(j-2)/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
    
        frozen_vertices=[ l, l+m1, l+m1+m2, l+m1+m2+m3 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    return r

def CartanMatrixSelfDefined(typ, rank):
#     if typ=='E' and rank==6:
#         r=Matrix([[2,0,-1,0,0,0],[0,2,0,-1,0,0],[-1,0,2,-1,0,0],[0,-1,-1,2,-1,0],[0,0,0,-1,2,-1],[0,0,0,0,-1,2]]) # this is the Cartan Matrix in Sage of type E6
#     else:
    
#         r = Matrix(rank, rank)
#         n = rank
#         for i in range(n):
#             if i + 1 <= n-1:
#                 r[i, i + 1] = -1
#             if 0 <= i - 1:
#                 r[i, i - 1] = -1
#             r[i, i] = 2

#         if typ == 'B' or typ == 2:
#             r[n-1, n - 2] = -2
#         elif typ == 'C' or typ == 3:
#             r[n - 2, n-1] = -2
#         elif typ == 'D' or typ == 4:
#             if n == 2:
#                 r[0, 1] = 0
#                 r[1, 0] = 0
#             elif 3 <= n:
#                 r[n - 3, n - 2] = -1
#                 r[n - 3, n-1] = -1
#                 r[n - 2, n - 3] = -1
#                 r[n-1, n - 3] = -1
#                 r[n - 2, n-1] = 0
#                 r[n-1, n - 2] = 0
#         elif typ == 'E' or typ == 5:
#             for k in [[2, 4], [4, 2]]:
#                 r[k[0], k[1]] = -1
#             for k in [[3, 4], [4, 3]]:
#                 r[k[0], k[1]] = 0
#         elif typ == 'F' or typ == 6:
#             r[1, 2] = -2
#         elif typ == 'G' or typ == 7:
#             r[0, 1] = -3
    
    if typ=='A' or typ==1:
        r=CartanMatrix(['A',rank])
    elif typ=='B' or typ==2:
        r=CartanMatrix(['B',rank])
    elif typ=='C' or typ==3:
        r=CartanMatrix(['C',rank])
    elif typ=='D' or typ==4:
        r=CartanMatrix(['D',rank])
    elif typ=='E' or typ==5:
        r=CartanMatrix(['E',rank])
    elif typ=='F' or typ==6:
        r=CartanMatrix(['F',rank])
    elif typ=='G' or typ==7:
        r=CartanMatrix(['G',rank])
     
    return r 

def IsALessOrEqualToBInRightAlignedOrder(a,b): # a, b are two segments
    r=0
    if a[1]<=b[1] and (a[1]<b[1] or a[0]<=b[0]):
        r=1
    
    return r
    
def merge_sortRightAlignOrder(array):
    # If the input array contains fewer than two elements,
    # then return it as the result of the function
    if len(array) < 2:
        return array

    midpoint = len(array) // 2

    # Sort the array by recursively splitting the input
    # into two equal halves, sorting each half and merging them
    # together into the final result
    return mergeRightAlignOrder(
        left=merge_sortRightAlignOrder(array[:midpoint]),
        right=merge_sortRightAlignOrder(array[midpoint:]))
    
def mergeRightAlignOrder(left, right):
    # If the first array is empty, then nothing needs
    # to be merged, and you can return the second array as the result
    if len(left) == 0:
        return right

    # If the second array is empty, then nothing needs
    # to be merged, and you can return the first array as the result
    if len(right) == 0:
        return left

    result = []
    index_left = index_right = 0

    # Now go through both arrays until all the elements
    # make it into the resultant array
    while len(result) < len(left) + len(right):
        # The elements need to be sorted to add them to the
        # resultant array, so you need to decide whether to get
        # the next element from the first or the second array
        if IsALessOrEqualToBInRightAlignedOrder(left[index_left],right[index_right])==1:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1

        # If you reach the end of either array, then you can
        # add the remaining elements from the other array to
        # the result and break the loop
        if index_right == len(right):
            result += left[index_left:]
            break

        if index_left == len(left):
            result += right[index_right:]
            break

    return result

def UnionOfTwoTableauxInMatrix(a,b):
    a1=MatrixToTableauTakeColumns(a)
    b1=MatrixToTableauTakeColumns(b)
    r1=UnionOfTwoTableaux(a1,b1)
    r=TableauToMatrix(r1)
    
    return r

def GVectorToModule_Yis(g,e1): # e1 is initial cluster
    r1=[]
    for i in [0..len(g)-1]:
        if g[i]>0:
            t1=power_of_monomial_Yis(e1[i],g[i])
            r1=r1+t1
    for i in [0..len(g)-1]:
        if g[i]<0:
            t1=power_of_monomial_Yis(e1[i],-g[i])
            r1=monomial_Yis_division(r1, t1)
    #print(r1)
    r=sorted(r1)
    
    return r

def FindNonRealModuleUsingQuiverWithMultipleArrows(k,n,l,e1,mutation_number): # l consists of a cluster, a quiver, and a labeled cluster, e1 is the set of initial cluster variables, e1 can choose a particular order, for example in Gr49, it can be e1=[[[1,2,3,5]],[[1,2,4,5]],[[1,3,4,5]],[[1,2,3,6]],[[1,2,5,6]],[[1,4,5,6]],[[1,2,3,7]],[[1,2,6,7]],[[1,5,6,7]],[[1,2,3,8]],[[1,2,7,8]],[[1,6,7,8]],[[1,2,3,4]],[[2,3,4,5]],[[3,4,5,6]],[[4,5,6,7]],[[5,6,7,8]],[[6,7,8,9]],[[1,2,3,9]],[[1,2,8,9]],[[1,7,8,9]]]
    r1=l
    typ=1
    rank=k 
    r2=r1[1] 
    
    r3=[]
    for i in range(len(r2)):
        for j in range(len(r2[0])):
            if abs(r2[i][j])>=2 and RemoveTrivialTableau2(r1[2][i][0],k,n)!=[] and RemoveTrivialTableau2(r1[2][j][0],k,n)!=[]:
                r3.append(sorted([i,j]))
                #print(i,j,r2[i][j],r1[2][i],r1[2][j])
    r3=removeDuplicatesListOfLists(r3)

    #print('r3',r3) 
    r=[]
    for multi_arrow_v in r3:
        #print('multi_arrow_v', multi_arrow_v)
        mat1=Matrix(r1[1])
        vertices1=[] 
        for i in r1[2]:
            t1=[1]+i
            vertices1.append(t1)
        vertices1=[vertices1, []]
        mutationSequence=multi_arrow_v*mutation_number

        for j in range(len(mutationSequence)): 
            #t1=vertices1[0][mutationSequence[j]][1]
            #print('t1', t1)
            #g2=GVectorOfTableau(k,n,t1,e1)[0]
            #print('g2', g2)
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
        #t1=vertices1[0][multi_arrow_v[1]][1]
        t1=vertices1[0][mutationSequence[j]][1]
        
        #print('t1', t1)
        
        g2=GVectorOfTableau(k,n,t1,e1)[0]
        
        p1=remove_common_factors_approximately(g2, mutation_number)
        
        p2=GVectorToTableau(p1,e1,k,n)
        #print('p2',p2)
        #print([multi_arrow_v, p2])
        r.append([multi_arrow_v, p2])
    return r

def FindNonRealModuleUsingQuiverWithMultipleArrows_Yis(typ,rank,L,l,e1,mutation_number): # C_l^{typ}, l \ge 0, L consists of a cluster, a quiver, and a labeled cluster, e1 is the set of initial cluster variables, e1 can choose a particular order, for example in Gr49, it can be e1=[[[1,2,3,5]],[[1,2,4,5]],[[1,3,4,5]],[[1,2,3,6]],[[1,2,5,6]],[[1,4,5,6]],[[1,2,3,7]],[[1,2,6,7]],[[1,5,6,7]],[[1,2,3,8]],[[1,2,7,8]],[[1,6,7,8]],[[1,2,3,4]],[[2,3,4,5]],[[3,4,5,6]],[[4,5,6,7]],[[5,6,7,8]],[[6,7,8,9]],[[1,2,3,9]],[[1,2,8,9]],[[1,7,8,9]]]
    r1=L 
    r2=r1[1] 
    IC=InitialCluster_Yis(typ,rank,l)
    cl=IC[2] # labels of cluster variables
    
    r3=[]
    for i in range(len(r2)):
        for j in range(len(r2[0])):
            if abs(r2[i][j])>=2 and IsInList(i,cl)==1 and IsInList(j,cl)==1:
                r3.append(sorted([i,j]))
                #print(i,j,r2[i][j],r1[2][i],r1[2][j])
    r3=removeDuplicatesListOfLists(r3)

    #print('r3',r3) 
    r=[]
    for multi_arrow_v in r3:
        #print('multi_arrow_v', multi_arrow_v)
        mat1=Matrix(r1[1])
        #print(mat1)
        vertices1=[] 
        #print('r1[2]',r1[2])
        for i in r1[2]:
            t1=[1]+i
            #print('t1',t1)
            vertices1.append(t1)
        vertices1=[vertices1, []]
        #print('vertices1', vertices1)
        mutationSequence=multi_arrow_v*mutation_number
        #print('mutation_sequence, l', mutationSequence, l)
        for j in range(len(mutationSequence)): 
            #print('mutation_sequence[j]', mutationSequence[j])
            #print('vertices1', vertices1)
            #t1=vertices1[0][mutationSequence[j]][1]
            #print('t1', t1)
            #g2=GVectorOfModule_Yis(t1,e1)[0]
            #print('g2', g2) 
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j], typ, rank, l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
        t1=vertices1[0][mutationSequence[j]][1]
        
        #print('t1', t1)
        
        g2=GVectorOfModule_Yis(t1,e1)[0]
        
        p1=remove_common_factors_approximately(g2, mutation_number)
        
        p2=GVectorToModule_Yis(p1,e1)
        #print('p2',p2)
        #print([multi_arrow_v, p2])
        r.append([multi_arrow_v, p2])
    return r

def FindNonRealModuleUsingQuiverWithMultipleArrows_Yis_print_g_vectors(typ,rank,L,l,e1,mutation_number): # C_l^{typ}, l \ge 0, L consists of a cluster, a quiver, and a labeled cluster, e1 is the set of initial cluster variables, e1 can choose a particular order, for example in Gr49, it can be e1=[[[1,2,3,5]],[[1,2,4,5]],[[1,3,4,5]],[[1,2,3,6]],[[1,2,5,6]],[[1,4,5,6]],[[1,2,3,7]],[[1,2,6,7]],[[1,5,6,7]],[[1,2,3,8]],[[1,2,7,8]],[[1,6,7,8]],[[1,2,3,4]],[[2,3,4,5]],[[3,4,5,6]],[[4,5,6,7]],[[5,6,7,8]],[[6,7,8,9]],[[1,2,3,9]],[[1,2,8,9]],[[1,7,8,9]]]
    r1=L 
    r2=r1[1] 
    IC=InitialCluster_Yis(typ,rank,l)
    cl=IC[2] # labels of cluster variables
    
    r3=[]
    for i in range(len(r2)):
        for j in range(len(r2[0])):
            if abs(r2[i][j])>=2 and IsInList(i,cl)==1 and IsInList(j,cl)==1:
                r3.append(sorted([i,j]))
                #print(i,j,r2[i][j],r1[2][i],r1[2][j])
    r3=removeDuplicatesListOfLists(r3)

    #print('r3',r3) 
    r=[]
    for multi_arrow_v in r3:
        #print('multi_arrow_v', multi_arrow_v)
        mat1=Matrix(r1[1])
        #print(mat1)
        vertices1=[] 
        #print('r1[2]',r1[2])
        for i in r1[2]:
            t1=[1]+i
            #print('t1',t1)
            vertices1.append(t1)
        vertices1=[vertices1, []]
        #print('vertices1', vertices1)
        mutationSequence=multi_arrow_v*mutation_number
        #print('mutation_sequence, l', mutationSequence, l)
        for j in range(len(mutationSequence)): 
            print('mutation_sequence[j]', mutationSequence[j])
            #print('vertices1', vertices1)
            t1=vertices1[0][mutationSequence[j]][1]
            #print('t1', t1)
            g2=GVectorOfModule_Yis(t1,e1)[0]
            print('g2', g2) 
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j], typ, rank, l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
        t1=vertices1[0][mutationSequence[j]][1]
        
        #print('t1', t1)
        
        g2=GVectorOfModule_Yis(t1,e1)[0]
        
        p1=remove_common_factors_approximately(g2, mutation_number)
        
        p2=GVectorToModule_Yis(p1,e1)
        #print('p2',p2)
        #print([multi_arrow_v, p2])
        r.append([multi_arrow_v, p2])
    return r
        

def KR_module_largest_s(i,s,k,typ,rank): # Y_{i,s}Y_{i,s-d}...
    d=distance_of_indices_KR_module_half(i,typ,rank)
    r=[]
    for j in [0..k-1]:
        r.append([i,s-2*j*d])
    return r



def UnionOfListsMany(l):
    r=[]
    for i in l:
        r=r+i
    return r

def GVectorOfModule_Yis(p1,r2): # p1 is a module in list form, r2 is initial cluster variables, in Gr48, r2=[[[1, 2, 3, 5]], [[1, 2, 4, 5]], [[1, 3, 4, 5]], [[1, 2, 3, 6]], [[1, 2, 5, 6]], [[1, 4, 5, 6]], [[1, 2, 3, 7]], [[1, 2, 6, 7]], [[1, 5, 6, 7]], [[1, 2, 3, 4]], [[2, 3, 4, 5]], [[3, 4, 5, 6]], [[4, 5, 6, 7]], [[5, 6, 7, 8]], [[1, 6, 7, 8]], [[1, 2, 7, 8]], [[1, 2, 3, 8]]],
    r1=removeDuplicatesListOfLists(UnionOfListsMany(r2)) # r1 are fundamental monomials
    m1=len(r1)
    m2=len(r2)
    gg=np.zeros((m1,m2), dtype=float)
    vv=[]
    #print(gg)
    for i in range(len(r1)):
        #print(r1[i])
        t1=p1.count(r1[i])
        vv.append(t1)
        for j in range(len(r2)):
            #print(r2[j])
            t1=r2[j].count(r1[i])
            gg[i, j]=t1
    vv=np.array(vv)
    #print('vv',vv)
    p6= np.linalg.lstsq(gg, vv, rcond=None)[0]

    #print(p6)
    t3=[]
    t6=[]
    for q in range(len(r2)):
        t2=int(round(p6[q]))
        t6.append([t2, r2[q]])
        t3.append(t2)
   
    r=[t3,t6]
    return r

# mutation of modules of quantum affine algebra of type Dn, C_l^g, l \ge 0, at the moment typ='D'
def computeEquationsForModulesYis(variable2, mat, k, typ, rank, l): # variable2=(variables on quiver, cluster variables obtained so far)
    
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            #print('i,variable[i]', i,variable[i])
            #print('power_of_monomial_Yis(variable[i][1], mat[i,k])', power_of_monomial_Yis(variable[i][1], mat[i,k]))
            newVariable=newVariable+power_of_monomial_Yis(variable[i][1], mat[i,k])
            #print('newVariable', newVariable)
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2=newVariable2 + power_of_monomial_Yis(variable[i][1], -mat[i,k])
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compare_weights_monomial_Yis(newVariable, newVariable2,typ,rank)
    
    #print('t1,newVariable,newVariable2,variable[k][1]',[t1,newVariableList,newVariable2List,variable[k][1]]) 

    if t1==1: 
        variable[k][1]=sorted(monomial_Yis_division(newVariable, variable[k][1]))
    else:
        variable[k][1]=sorted(monomial_Yis_division(newVariable2, variable[k][1])) 
        
    clusterVariables=variable[k][1]
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def remove_common_factors_approximately(l, a): # l[i]/t, t>a
    r1=copy.deepcopy(l)
    r2=list(map(lambda x: abs(x), r1))
    r3=removeDuplicatesListOfLists(sorted(SetDifference(r2, [0])))
    minm=max(r3)
    for i in r3:
        if i<minm and i>a:
            minm=i
        
    #print('minm', minm)
    
    t1=[]
    for i in r1:
        t1.append(round(i/minm, 6))
    r0=t1
    
    multi=1
    while 1:
        r1=[i*multi for i in r0]
        #print(r1)
        r=[]
        for i in r1:
            t1=round(i, 0)
            t2=round(i, 1)
            #print(t1,t2,abs(t1-t2))
            if abs(t1-t2)<0.00001:
                r.append(t1)
            else:
                break
        if len(r)==len(r1):
            break
        else:
            multi=multi+1
    
    return r

def distance_of_indices_KR_module_half(i,typ,rank): # we use self-defined Cartan matrices, can change to Sage convention later
    r=1
    if typ=='G' and i==2:
        r=3
    elif typ=='B' and i!=rank:
        r=2
    elif typ=='C' and i==rank:
        r=2
    elif typ=='F' and (i==1 or i==2):  # this corresponds to self-defined Cartan matrix of type F4, can change to Sage convention later
        r=2
    return r
            
def KR_module(i,s,k,typ,rank): # Y_{i,s}Y_{i,s+d}...
    d=distance_of_indices_KR_module_half(i,typ,rank)
    r=[]
    for j in [0..k-1]:
        r.append([i,s+2*j*d])
    return r

def ModuleToMultiSegment(Y,l): # l is a list, l[i] is the index of some Y[i,j]
    #print(l)
     
    r=TranslateModuleToMultiSegmentsMonomial(Y,l)

    return r

def TranslateModuleToMultiSegmentsMonomial(Y,l): # l is a list, l[i] is the index of some Y[i,j]
 
    r=[]
    for j in l:
        #print(j)
        r=append(r, TranslateQuantumAffineToLadders(j))
        
    
    #r=SortBySecondTerms(r)
    r=merge_sortRightAlignOrder(r)
    r=list(reversed(r))

    return r


def TranslateQuantumAffineToLadders(l):
    i=l[0]
    s=l[1] 
    r=[(s-i)/2+1/2, (s+i)/2-1/2]

    r=[r[0]+1/2, r[1]+1/2]

    return r


def SortBySecondTerms(l):

    r=[]

    if l!=[]:
        #print(l)

        t1=[]
        for i in l:
            t1=append(t1, i[1])

        t1=sorted(t1)

        #print(t1)

        t2=sorted(l)

        r=[] 
        for i in reversed([min(t1)..max(t1)]):
            for j in t2:
                if j[1]==i:
                    r=append(r, j)
    return r


def ModuleToTableauInMatrix(Y,l,k,n):
    t1=ModuleToMultiSegment(Y,l)
    r=MultisegmentToTableauGrassmannian(k,n,t1)
    return r

def ModuleToTableau(Y,l,k,n):
    t1=ModuleToTableauInMatrix(Y,l,k,n)
    r=MatrixToTableauTakeColumns(t1)
    return r

def MultisegmentToTableauGrassmannianOneSegment(k, m): # Gr(k,n)
 
    a=m[0]
    b=m[1]

    r=Matrix(k,1)
    r1=SetDifference(ListAToN(1-a, 1+k-a), k-b)
    for i in [0..k-1]: 
        r[i,0]=r1[i]

    return r


def MultisegmentToTableauGrassmannian(k,n, l): # Gr(k,n)

    r=0
    if l!=[]:
        r=MultisegmentToTableauGrassmannianOneSegment(k,l[0])
        for i in [1..len(l)-1]:
            t1=MultisegmentToTableauGrassmannianOneSegment(k, l[i])
            r=UnionOfTwoTableauxInMatrix(r, t1)
             
        r=RemoveTrivialTableauInMatrix(r,k,n) 
    return r

def RemoveTrivialTableauInMatrix(a1, n,m): # n is the row of tableau, m can take sufficient large

    t1=MatrixToTableauTakeColumns(a1)
    t2=RemoveTrivialTableau(t1,n,m)
    r=TableauToMatrix(t2)

    return r

def TableauInMatrixToMultiSegments(Y,a): # a is a tableau in matrix   
    k=a.nrows()
    b=MatrixToTableauTakeColumns(a)
    a1=TableauToSmallGapTableau(k,b);
    #print(k,a1)
    r1=CollectFirstElementInSmallGapTableau(a1);
    r2=CollectMissingElementInSmallGapTableau(a1)
    r=[]
    for i in [0..len(r1)-1]:
        t1=[1-r1[i], k-r2[i]]
        r.append(t1)
    return r

def TableauToMultiSegments(Y,l): # l is a tableau T, l[i] is column of i of T, l=[] means trivial tableau, corresponding to trivial module \mathbb{C}
 
    if l==[]:
        r=[]
    else:
        t2=TableauToMatrix(l)
        r=TableauInMatrixToMultiSegments(Y,t2)

    return r

def ProductOfElementsInList(l):
    r=1
    for i in l:
        r=r*i
    return r

def TableauToModulePolynomialOfTableauxWithCoeff(Y,l): #
 
    r=0
    for i in l:
        if i[0]!=0 and i[1]!=[]:
            t1=TableauToModuleHasY(Y,i[1])
            #print(t1)
            r=r+i[0]*ProductOfElementsInList(t1)
       
    return r

def TableauToModulePolynomialOfTableauxWithCoeffHasNoY(Y,l): #
 
    r=[]
    for i in l:
        if i[0]!=0 and i[1]!=[]:
            t1=TableauToModule(Y,i[1])
            #print(t1)
            r.append([i[0], t1])
       
    return r


def TableauToModule(Y,l): # l is a tableau T, l[i] is column of i of T
 
    t1=TableauToMultiSegments(Y,l)
    r=MultiSegmentToModule(t1)

    return r

def TableauToModulePolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModule(Y,i[1])]) 

    return r

def TableauToModuleHasY(Y,l): # l is a tableau T, l[i] is column of i of T
 
    t1=TableauToModule(Y,l)
    r=[]
    for i in t1:
        r.append(Y[i[0],i[1]])

    return r

def TableauToModulePolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleHasY(Y,i[1])]) 

    return r


def TableauInMatrixToModule(Y,l): # l is a tableau T in terms of matrix, l=[] means l is trivial tableau
    
    if l==[]:
        r=[]
    else:

        t1=TableauInMatrixToMultiSegments(Y,l)
        r=MultiSegmentToModule(t1)

    return r

def TableauToModuleInMatrixPolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrix(Y,i[1])]) 

    return r

def TableauToModuleInMatrixPolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrixHasY(Y,i[1])]) 

    return r


def TableauToModuleInMatrix(Y,l): # l is a tableau T in terms of matrix, l=[] means l is trivial tableau
    if l==[]:
        r=[]
    else:
    
        r=TableauInMatrixToModule(Y,l)

    return r

def TableauToModuleInMatrixHasY(Y,l): # l is a tableau T
 
    r1=TableauToModuleInMatrix(Y,l)
    r=[]
    for i in r1:
        r.append(Y[i[0],i[1]])

    return r


def TableauToModuleInMatrixPAdicConvention(Y,l): # l is a tableau T
 
    r1=TableauToModuleInMatrix(Y,l)
    r=[]
    for i in r1:
        t1=i
        r=append(r, [t1[0], -t1[1]-1]) 

    return r

def TableauToModulePAdicConvention(Y,l): # l is a tableau T
 
    r1=TableauToModule(Y,l)
    r=[]
    for i in r1:
        t1=i
        r=append(r, [t1[0], -t1[1]-1]) 

    return r

def TableauToModulePAdicConventionPolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModulePAdicConvention(Y,i[1])]) 

    return r

def TableauToModuleInMatrixPAdicConventionPolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrixPAdicConvention(Y,i[1])]) 

    return r


def TableauToModuleInMatrixPAdicConventionHasY(Y,l): # l is a tableau T
    
    if l==[]:
        r=[]
    else:
     
        r1=TableauToModuleInMatrix(Y,l)
        r=[]
        for i in r1:
            t1=i
            r=append(r, Y[t1[0], -t1[1]-1]) 

    return r

def TableauToModulePAdicConventionHasY(Y,l): # l is a tableau T
    
    if l==[]:
        r=[]
    else:
     
        r1=TableauToModule(Y,l)
        r=[]
        for i in r1:
            t1=i
            r=append(r, Y[t1[0], -t1[1]-1]) 

    return r

def TableauToModuleInMatrixPAdicConventionPolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrixPAdicConventionHasY(Y,i[1])]) 

    return r

def TableauToModulePAdicConventionPolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModulePAdicConventionHasY(Y,i[1])]) 

    return r

def SegmentToModule(l): 
    
    a=l[0]
    b=l[1]
     
    r=[b-a+1,a+b-1]
        
    
    return r

def MultiSegmentToModule(l):  
    
    r=[]
    for i in l:
        r.append(SegmentToModule(i))
        
    
    return r

def ProductOfExponentsInTermsOfListMonomial(a,b): 
    #print('a,b',a,b)
    t0=a[0]*b[0]
    t1=AddVlueOfTwoLists(a[1],b[1]) 
    r=[t0, t1]
    return r

def ProductOfExponentsInTermsOfListPolynomial(a,b): 
    r=[]
    #print('a,b',a,b)
    for i in a:
        for j in b:
            r.append(ProductOfExponentsInTermsOfListMonomial(i,j))
    return r

def combineLikeTerms3(l):

    r=[]
    l2=l
    
    #print('l',l)
    temp2=[]

    for i in l2:

        if (IsInList(removeAnElementInList(0,i), temp2)==0): 
            t=0
            for j in l2:
                if removeAnElementInList(0,j)==removeAnElementInList(0,i):
                    t=t+j[0]
              
            s=i
            s[0]=t
            r.append(s)
            temp2.append(removeAnElementInList(0,i))
    
    r1=r
    r=[]
    for i in r1:
        if i[0]!=0:
            r.append(i)

    return r

def find_gcd(list):
    x = reduce(gcd, list)
    return x

def IntersectionOfLists(a,b):
    r=[x for x in a if x in b]
    return r

def IntersectionOfLists2(a,b):
 
    a1=ChangeListOfMatricesToSetOfMatrices(a)
    b1=ChangeListOfMatricesToSetOfMatrices(b)
    r=list(a1.intersection(b1))

    return r

def IntersectionOfLists3(a,b):
    
    r=list(set(a) & set(b))

    return r

def FindAllPrimeTableauxParallel(fp,T,k,n): # T[i] is a row
    r=[]
    i=TransposeTableauSLk(T) # i[j] are columns
    #print(i)
    if QaRootsOfTableaux(k, TableauToMatrix(i))<=2:
        if IsTableauPrime(i,n)==1:
            r=[i]
            #print(r)
            F1=open(fp, 'a+')
            F1.write(str(i)+'\n')
            F1.close()
    return r


def list_of_tuples_to_list_of_lists(l):
    r = [list(elem) for elem in l]
    return r

def list_nonsemistandard_tableaux_to_semistandard(l):
    r=[]
    for i in l:
        t1=SortRowsOfTableau(i)
        r.append(t1)
    return r

def generate_all_semistandard_tableaux(k,n,m): # k rows, m columns, max = n, r[i][j] is a row in tableau r[i]
    t1=[]
    for i in [0..k-1]:
        t1.append(m)
    r=SemistandardTableaux(t1, max_entry=n)
    return r

def IsSemistandardTableauInRows(a): # a[i]'s are rows
    r=1
    
    for i in [0..len(a)-2]:
        if len(a[i])<len(a[i+1]):
            r=0
            return r
    for i in [0..len(a)-1]:
        for j in [0..len(a[i])-2]:
            if a[i][j+1]<a[i][j]:
                r=0
                return r
    r1=TransposeTableauSLk(a)
    for i in [0..len(r1)-1]:
        for j in [0..len(r1[i])-2]:
            if r1[i][j+1]<=r1[i][j]:
                r=0
                return r
        
    return r
    
def TransposeTableauSLk(a): # a[i] are columns
    r=[]
    for j in [0..len(a[0])-1]:
        t1=[]
        for i in [0..len(a)-1]:
            if j<len(a[i]):
                t1.append(a[i][j])
        r.append(t1)

    return r

def AreAllTableauxWithQLessOrEqual2(l): # l[i] is a list, l[i][j] are columns
    r=1
    k=len(l[0][0])
    for i in l:
        i1=TableauToMatrix(i)
        t1=QaRootsOfTableaux(k, i1)
        if t1>2:
            r=0
            return r
    return r

def IsCompatiblePairwiseTableaux(l, n): # l[i] is a tableau, l[i][j] is a column
    r=1
    r1=NchooseM(l,2)
    for r2 in r1:
        t1=TableauToMatrix(r2[0])
        t2=TableauToMatrix(r2[1])
        p1=IsCompatiblePairTableauxInMatrix(t1,t2,n)
        if p1==0:
            r=0
            return r
    return r

def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller
        
def FindAllPrimeTableaux(fp,k,n,m): # r[i][j] are columns in r[i]
    r1=generate_all_semistandard_tableaux(k,n,m)
    r=[] 
    for i1 in r1:
        i=TransposeTableauSLk(i1) # i[j] are columns
        #print(i)
        if QaRootsOfTableaux(k, TableauToMatrix(i))<=2:
            if IsTableauPrime(i,n)==1:
                r.append(i)
                print(i)
                F1=open(fp, 'a+')
                F1.write(str(i)+'\n')
                F1.close()
                
    return r 

class BipartiteGraphMaxMatching:
    def __init__(self,graph): # graph is given by list of lists, it is a bipartite graph, graph[i][j]=1 means xi is connected to yj 
         
        # residual graph
        self.graph = graph
        self.ppl = len(graph)
        self.jobs = len(graph[0])
 
    # A DFS based recursive function
    # that returns true if a matching
    # for vertex u is possible
    def bpm(self, u, matchR, seen):
 
        # Try every job one by one
        for v in range(self.jobs):
 
            # If applicant u is interested
            # in job v and v is not seen
            if self.graph[u][v] and seen[v] == False:
                 
                # Mark v as visited
                seen[v] = True
 
                '''If job 'v' is not assigned to
                   an applicant OR previously assigned
                   applicant for job v (which is matchR[v])
                   has an alternate job available.
                   Since v is marked as visited in the
                   above line, matchR[v]  in the following
                   recursive call will not get job 'v' again'''
                if matchR[v] == -1 or self.bpm(matchR[v], matchR, seen):
                    matchR[v] = u
                    return True
        return False
 
    # Returns maximum number of matching
    def maxBPM(self):
        '''An array to keep track of the
           applicants assigned to jobs.
           The value of matchR[i] is the
           applicant number assigned to job i,
           the value -1 indicates nobody is assigned.'''
        matchR = [-1] * self.jobs
         
        # Count of jobs assigned to applicants
        result = 0
        for i in range(self.ppl):
             
            # Mark all jobs as not seen for next applicant.
            seen = [False] * self.jobs
             
            # Find if the applicant 'u' can get a job
            if self.bpm(i, matchR, seen):
                result += 1
        return result

def matrix_from_edge_set_of_bipartite_graph(l,n,m): # here n=len(l), l = [[2,4,5],[2,3],...] means vertex x1 connect to y2, y4, y5, vertex x2 connect to y2, y3 
    #print('l',l)
    r=[]
    for i in [0..n-1]:
        t1=[]
        for j in [0..m-1]:
            t1.append(0)
        r.append(t1)
    for i in [0..len(l)-1]:
        for j in [0..len(l[i])-1]:
            r[i][l[i][j]]=1
    return r

def IsInList(i,l):
    if (i in l)==True:
        r=1
    else:
        r=0
    return r

def NonCrossingTupleWithSameContentAsT_new(T): 
    if T and T[0]:
        r=list(candidates_new(Counter(sorted(flatten(T))), len(T[0])))
        r=removeDuplicatesListOfLists(r)
        return r

def NonCrossingTupleWithSameContentAsT_new_smaller_or_equal_weight(T, typ,rank): 
    r1=NonCrossingTupleWithSameContentAsT_new(T)
    r=ChooseTableauxWithSmallOrEqualWeight(T,r1,typ,rank)
    return r    

def candidates_new(content, k, candidate=()): 
    if len(content) < k:
        if len(content) == 0 and candidate:
            yield candidate
        else:
            return
        
    for I in itertools.combinations(content, k):
        # tuples in candidate are in non-decreaseing order
        if candidate and I < candidate[-1]:
            continue
            
        # new tuple is non-crossing with the other tuples
        if all(is_noncrossing_new(I, J) for J in candidate):
            yield from candidates_new(content - Counter(I), k, candidate + (I,))
            
@functools.lru_cache()  
def is_noncrossing_new(I,J): # I,J have the same size
    k = len(I)

    for a in range(k - 1):
        for b in range(a + 1, k):
            if (I[a+1:b] == J[a+1:b] and
                    not is_weakly_separated(I[a:b+1], J[a:b+1])):
                return False

    return True
    
def is_weakly_separated(I,J): # I,J have the same size
    r1 = sorted(set(I).difference(J))
    r2 = sorted(set(J).difference(I))

    if not r1 or not r2:
        return True

    if r1[0] >= r2[0]:
        r1, r2 = r2, r1
 
    for i in range(len(r1) - 1):
        if r2[0] < r1[i+1]: 
            return r2[-1] <= r1[i+1]
 
    return r1[-1] <= r2[0]


def InitialClusterWithoutNumericalMatrices(rank,n): # Gr(rank,n)
    sizeColumn=n-rank
    k=sizeColumn    
    k1=rank
    p1=k*rank+1
    mat=Matrix(p1,p1)
    for i in range(p1): 
        i1=i+1
        if i1==1: 
            mat[i,i+k+1]=1
            mat[i, p1-1]=1
            mat[i,i+k]=-1
            mat[i, i+1]=-1
        elif i1>=2 and i1<=k-1: 
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1==k: 
            mat[i,i-1]=1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+k+1]=1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k >=2 and i1 % k<=k-1:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1>=2*k and i1<=(rank-1)*k and i1 % k==0:
            mat[i,i-k-1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1>(rank-1)*k and i1<p1 and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+1]=-1
        elif i1>=(rank-1)*k+2 and i1<rank*k:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==rank*k:
            mat[i,i-k-1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==p1:
            mat[i,0]=-1

    vertices0=[]

    for j in range(k1-1,-1,-1):
        for i in range(k1,k1+k):
            t1=list(range(1,j+1))
            t2=list(range(i-k1+j+2,i+2))
            t3=t1+t2
            vertices0.append(t3)

    vertices0.append(list(range(1,k1+1)))  
    verticesTableaux = [] # Tableaux are represented by matrices
    for i in range(len(vertices0)):
        verticesTableaux.append([0, [vertices0[i]], i]) # [vertices0[i]] is an one column tableau
    mat1 = Matrix(p1,p1)
    for i in range(p1):
        for j in range(p1):
            mat1[i,j]=mat[i,j]
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r

def EvacuationOfTableauNTimesTheNthTisListOfLists_List(N,l,n): # l is a list, l[i] is a tableau written as list of lists
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesTheNthTisListOfLists(N,i,n)
        r.append(t1)
    return r

def EvacuationOfTableauNTimesTheNthTisListOfLists_ListLabeled(N,l,n): # l is a list, l[i]=[T, i], T is a tableau written as list of lists
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesTheNthTisListOfLists(N,i[0],n)
        r.append([t1,i[1]])
    return r

def PromotionOfTableauNTimesTheNthTisListOfLists_List(N,l,n): # l is a list, l[i] is a tableau written as list of lists
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesTheNthTisListOfLists(N,i,n)
        r.append(t1)
    return r

def PromotionOfTableauNTimesTheNthTisListOfLists_ListLabeled(N,l,n): # l is a list, l[i]=[T, i], T is a tableau written as list of lists
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesTheNthTisListOfLists(N,i[0],n)
        r.append([t1,i[1]])
    return r


def TakeIthElements(i,l):
    r=[]
    for i1 in l:
        r.append(i1[i-1])
    return r



def removeAnElementInListList(i, l):
 
    r=[]
    for j in l:
        r.append(removeAnElementInList(i, j))
    
    return r

def EvacuationOfPolynomialInPluckerNTimesListForPairList(N,ll,n): 
    r=[]
    for i in ll:
        t1=EvacuationOfPolynomialInPluckerNTimesListForPair(N,i,n)
        r=r+t1
        
    return r

def PromotionOfPolynomialInPluckerNTimesListForPairList(N,ll,n): 
    r=[]
    for i in ll:
        t1=PromotionOfPolynomialInPluckerNTimesListForPair(N,i,n)
        r=r+t1
    
    return r

def PluckerPolynomialTableauExpansionsList(ll,n):
    r=[]
    for i in ll:
        t1=PluckerPolynomialTableauExpansions(i,n)
        r=r+t1
    return r

def ShiftModN(l,a,n): #  l is sent to l+a mod n
    r=Mod(l+a, n)   # range of Mod() is from 0 to n-1
    return r

def ShiftModNKTimes(l,a,n,k):  
    r=l
    r=ShiftModN(r, k*a, n) 
    return r

def CrossProduct(l): # l=[p1,p2,...,pk], cross product of p1,p2,...,pk, pi are k+1 \times 1 matrices

    k=len(l)
    n=k+1
    #r=Matrix(n,1)
    r=np.zeros((n,1), dtype=float)

    #x=Matrix(k+1,k)
    x=np.zeros((k+1,k), dtype=float)
    
    for i in [0..k]:
        for j in [0..k-1]:
            x[i,j]=l[j][i,0]
    #print('x',x)

    for i in [1..n]:
        t1=SetDifference(ListAToN(1,n), [i])
        t2=ListAToN(1,k)
        #print('x,i,t1,t2',x,i,t1,t2,(-1)^(i-1)*Minor(x,t1,t2)) 
        r[i-1,0]=(-1)^(i-1)*Minor(x,t1,t2)
    
    return r

def MarshScottTwistHalf(i1,p): # p is a k times n matrix
 
    k,n=p.shape
    #print(k,n)
    P=[]
    for j in range(1,k): 
        t1=np.zeros((k+1,1), dtype=float)
        #print(t1)
        t2=ShiftModNKTimes(i1, -1, n, k-j)
        #print('t2', t2)
        for d in range(k):
            t1[d,0]=p[d,t2] 
        #print('t1',t1)
        P.append(t1) 
 
    r=CrossProduct(P) 

    return r

def MarshScottTwist(p): # p is a k times n matrix, p is np.array([[],[]])
    #print(p) 
    k,n=p.shape
    #print(k,n)

    r=np.zeros((k,n), dtype=float)
    #r=Matrix(k,n)
    #print(r)

    for i in range(n):
        #print(i, p)
        t1=MarshScottTwistHalf(i,p) 
        #print('t1',t1)
        for d in [0..k-1]: 
            r[d,i]=t1[d,0] 
            
    return r
     
def merge_sort(array):
    # If the input array contains fewer than two elements,
    # then return it as the result of the function
    if len(array) < 2:
        return array

    midpoint = len(array) // 2

    # Sort the array by recursively splitting the input
    # into two equal halves, sorting each half and merging them
    # together into the final result
    return merge(
        left=merge_sort(array[:midpoint]),
        right=merge_sort(array[midpoint:]))
    
def merge(left, right):
    # If the first array is empty, then nothing needs
    # to be merged, and you can return the second array as the result
    if len(left) == 0:
        return right

    # If the second array is empty, then nothing needs
    # to be merged, and you can return the first array as the result
    if len(right) == 0:
        return left

    result = []
    index_left = index_right = 0

    # Now go through both arrays until all the elements
    # make it into the resultant array
    while len(result) < len(left) + len(right):
        # The elements need to be sorted to add them to the
        # resultant array, so you need to decide whether to get
        # the next element from the first or the second array
        if left[index_left] <= right[index_right]:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1

        # If you reach the end of either array, then you can
        # add the remaining elements from the other array to
        # the result and break the loop
        if index_right == len(right):
            result += left[index_left:]
            break

        if index_left == len(left):
            result += right[index_right:]
            break

    return result

def CrossProductResultInMinorIndices(l): # l=[l_1,l_2,l_3,...,l_k], l_i is an integer, denote the l_i-th column of a matrix 
    k=len(l)
    n=k+1
    r=[]
    
    for i in [1..n]:
        t1=SetDifference(ListAToN(1,n), [i])
        r.append([(-1)^(i-1), t1, l])
    
    return r

def MarshScottTwistHalfResultInMinorIndices(i1,k,n):  

    P=[]
    for j in range(1,k):  
        t2=ShiftModNKTimes(i1, -1, n, k-j)+1
        t2=ModResultOneToN(t2,n)
        P.append(t2)
    P1=[]
    for i in [1..n]:
        if IsInList(i, P)==1:
            P1.append(i)
    r=CrossProductResultInMinorIndices(P1) 

    return r

def MarshScottTwistResultInMinorIndices(k,n): 
    r=[]
    for i in [1..k]:
        t1=[]
        for j in [1..n]:
            t1.append(0)
        r.append(t1)      # r[i] are rows of r, r is a k by n matrix written in terms of list of lists
    
    for i in range(n): 
        t1=MarshScottTwistHalfResultInMinorIndices(i,k,n) 
        #print('t1',t1)
        for d in [0..k-1]: 
            r[d][i]=t1[d] 
            
    return r

def TransposeOfRectangularTableau(l): # l=[l_1, l_2, ...], l_i is a list
    r=[]
    if l!=[]:
        n=len(l)
        m=len(l[0])
        for j in [0..m-1]:
            t1=[]
            for i in [0..n-1]:
                t1.append(l[i][j])
            r.append(t1)
    return r

def ContentOfMarshScottTwistOfPlucker(l,k,n): # l=P=[a,b,c,d] are indices of a Plucker coordinate, Grkn, return content of tau(P)
    r1=MarshScottTwistResultInMinorIndices(k,n)
    r=[]
    for i in [0..k-1]: 
        r=r+r1[i][l[i]-1][2]
    r.sort()
    
    return r


def IsTotallyPositive(g):
    r=1

    n=g.nrows()
    m=g.ncols()

    for i in [1..min(n,m)]:
        for t1 in NchooseM(ListAToN(1,n), i):
            for t2 in NchooseM(ListAToN(1,m), i):
                t3=Minor(g, sorted(t1), sorted(t2))
                
                if t3<=0:
                    print(t3,sorted(t1),sorted(t2))
                    r=0 
                    break

    return r 

def IsTotallyNonNegative(g):
    r=1

    n=g.nrows()
    m=g.ncols()

    for i in [1..min(n,m)]:
        for t1 in NchooseM(ListAToN(1,n), i):
            for t2 in NchooseM(ListAToN(1,m), i):
                t3=Minor(g, sorted(t1), sorted(t2))
                if t3<0:
                    r=0 
                    break
    return r


def Multiplyq2(l,q):
 
    r=[]
    for i in l:
        temp=i
        temp[0]=simplify(q*temp[0])
        r.append(temp)
 
    return r

def Multiplyq2NoChangeValue(l,q):
 
    r=[]
    for i in l:
        t1=[simplify(q*i[0]), i[1]]
        r.append(t1)
 
    return r

def PluckersWhichAreOne2(n,n1,m): # also remove 189, 129 in Gr39, n is number of rows of tableau, Gr(n,n1)
  
    r=[]
    for i in [1..m]: 
        t1=[]
        for j in [i..n+i-1]:
            t1=append(t1, j)
             
        r=append(r, t1)
         

    for j in [1..n]:
        t1=[]
        for i in [1..j]:
            t1=append(t1, i)
             
        for s in [n1-n+j+1..n1]:
            t1=append(t1, s)
             
        r=append(r, t1)
         
    return r

def RemoveTrivialTableau2(a1, n,n1): # n is the row of tableau, also remove 189, 129 in Gr39, Gr(n,n1),m1 =m= n1-n+1
 
    r=a1
    m=n1-n+1
    t2=PluckersWhichAreOne2(n,n1,m)

    #print(t2)

    sn=1

    while sn!=0:

        r1=r
        for i in [0..len(t2)-1]:

            #print(t2[i],r)

            if r==[]:
                return r
            elif IsTableauAFactorOfB([t2[i]], r)==1:
                r=TableauDivision(r, [t2[i]])
                #print(r)
                  
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1  

    return r

def PluckersWhichAreOne(n,m):
    r=[]
    for i in range(1,m+1):
        t1=[]
        for j in range (i,n+i):
            t1.append(j)
        r.append(t1)
    return r

def RemoveTrivialTableau(a1, n,m): # n is the row of tableau, m can take sufficient large
    r=a1
    t2=PluckersWhichAreOne(n,m);
    sn=1
    while sn!=0:
        r1=r;
        for i in range(len(t2)):
            #print(r, t2[i])
            if r==[]:
                break
            elif IsTableauAFactorOfB([t2[i]], r)==1:
                
                r=TableauDivision(r, [t2[i]]);
                
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1
        
    
    return r

def IsSubsetAsMultiset(a,l):
    r=1
    for i in a:
        t1=NumberOfA(a,i)-NumberOfA(l,i)
        #print(t1)
        if t1>0:
            r=0
            break
    return r

def IsTableauAFactorOfB(a,b):
    t1=TableauToMatrix(a);
    t2=TableauToMatrix(b);
    r1=MatrixTakeRows(t1);
    r2=MatrixTakeRows(t2);

    r=1;
    for i in range(len(r1)):
        if IsSubsetAsMultiset(r1[i], r2[i])==0:
            r=0
            break
    return r

def NumberOfA(l,a):
    r=0;
    for i in l:
        if i==a:
            r=r+1
    return r

def IsCompatiblePairTableauxInMatrix(T1, T2,n):
 

    r=0

    if IsLCTTPrimeNew(T1, T2,n)==1 and IsLCTTPrimeNew(T2, T1,n)==1:
        r=1 

    return r 

def IsLCTTPrimeNew(T1, T2,n):
 
    g1=TableauToSmallGapTableauSeparated(T1)
    g2=TableauToSmallGapTableauSeparated(T2)

    #print(g1,g2)

    r=0

    a=TheSetXTTPrime(T1, T2)
    b=TheSetYTTPrime(T1, T2,n)

    #print(a,b)

    #print(lengthOfList(a),lengthOfList(b))
    if len(a)>len(b):
        r=0
    elif len(a)==0:
        r=1
    else:

        r1=[]
        for pp1 in [0..len(a)-1]:
            p1=a[pp1]
            t1=[]
            for pp2 in [0..len(b)-1]:
                p2=b[pp2]
                i=p1[0]-1
                j=p1[1]-1
                i1=p2[0]-1
                j1=p2[1]-1
                if ( i==i1 and IsWeaklySeparated(g2[j1], g2[j])==0 and g2[j][0,0]<g2[j1][0,0] ) or ( j==j1 and IsWeaklySeparated(g1[i], g1[i1])==0 and g1[i1][0,0]<g1[i][0,0] ):
                    t1.append(pp2)
            
            if t1==[]: # can speed up the computations
                r=0
                return r
                break
            r1.append(t1)

        #print(r1)

        #r3=CartesianProductOfListsInAList(r1)
        #print('len(r3)', len(r3))
        #print(r3)

#         for i in r3:
#             #print(i)
#             t1=removeDuplicatesListOfLists(list(i))
#             #print(i,t1)
#             if len(t1)==len(a):
#                 #print(t1)
#                 r=1
#                 return r
#                 break
        bipartite=matrix_from_edge_set_of_bipartite_graph(r1,len(a),len(b))
        g=BipartiteGraphMaxMatching(bipartite)
        num=g.maxBPM()
        if num==len(a):
            r=1
        else:
            r=0

    return r 
    
def TheSetXTTPrime(T1, T2):

    t1=TableauToSmallGapTableauSeparated(T1)
    t2=TableauToSmallGapTableauSeparated(T2)

    #print(t1,t2)

    r=[]
    for i in [0..lengthOfList(t1)-1]: 
        for j in [0..lengthOfList(t2)-1]: 
            r1=MatrixTakeColumns(t1[i])[0]
            r2=MatrixTakeColumns(t2[j])[0]
            if IsWeaklySeparated(r1,r2)==0 and t2[j][0,0]<t1[i][0,0]:
                r=append(r, [i+1,j+1])

    return r 
    
def TheSetYTTPrime(T1, T2,n):
 
    t1=TableauToSmallGapTableauSeparated(T1)
    t2=TableauToSmallGapTableauSeparated(T2)

    #print(t1,t2)

    r=[]
    for i in [0..lengthOfList(t1)-1]: 
        for j in [0..lengthOfList(t2)-1]:
            r0=t1[i]
            #print('r0',r0)
            g1=SemistandardTableau(MatrixTakeRows(r0))
            #print('g1',g1)
            r1=g1.promotion(n)
            r1=MatrixTakeColumns(Matrix(r1))[0]
            r2=MatrixTakeColumns(t2[j])[0]
            #print('r1,r2',r1,r2)
            if IsWeaklySeparated( r1, r2 )==0 and t2[j][0,0]<=r0[0,0]:
                r=append(r, [i+1,j+1])

    return r 
    
def CartesianProductOfListsInAList(l): # l=[l1,l2,...], l1 is a set written in terms of a list
 
    r=[]
    for i in itertools.product(*l):
        r.append(i)

    return r 

def MatrixToTableauTakeColumnsNPArray(a):  # a is np array
 
    t1=list(a.shape)
    n=t1[0]
    m=t1[1]
    #print(n,m,a)
    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            t2.append(a[j,i])
        
        r.append(t2)
   

    return r

def MatrixToTableauTakeColumns(a): 
 

    n=a.nrows()
    m=a.ncols()

    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            t2=append(t2,a[j,i])
        
        r=append(r, t2)
   

    return r

def SortEachRowOfMatrix(l):

    r=[]
    for i in [0..l.nrows()-1]: 
        r1=[]
        for j in [0..l.ncols()-1]:
            r1=append(r1, l[i,j])
             
        r=append(r, sorted(r1))
        

    r=Matrix(r)

    return r 

def SortEachRowOfMatrixList(l):
 
    r=[]
    for i in l:
        r=append(r, SortEachRowOfMatrix(i)) 

    return r 

def CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n): 
 
 
    t1=NchooseM(ListAToN(1,n),k)
    t2=[]
    for i in t1:
        t2=append(t2, sorted(i))
         

    r=t2

    return r 

def CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJCasesFaster(N, k,n): # compute n-tuples of plucker which are compatible, the n-tuple can be (I, I, J), (I, I, I).


    if N==1:

        r=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)

    elif N==2:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)

        r1=[]
        for i in [0..lengthOfList(t3)-1]:
            for j in [i..lengthOfList(t3)-1]:
                if IsWeaklySeparated(t3[i],t3[j])==1:
                    #print(t3[i],t3[j])
                    r1=append(r1, [t3[i],t3[j]])
                     
        r=r1


    elif N==3:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1  and IsWeaklySeparated(t3[i1],t3[i3])==1:
                        r1=append(r1, [t3[i1],t3[i2],t3[i3]])
                         

        r=r1

    elif N==4:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    for i4 in [i3..lengthOfList(t3)-1]:
                        if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1  and IsWeaklySeparated(t3[i3],t3[i4])==1 and IsWeaklySeparated(t3[i1],t3[i3])==1 and IsWeaklySeparated(t3[i1],t3[i4])==1 and IsWeaklySeparated(t3[i2],t3[i4])==1:
                            r1=append(r1, [t3[i1],t3[i2],t3[i3],t3[i4]])
                         

        r=r1

    elif N==5:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    for i4 in [i3..lengthOfList(t3)-1]:
                        for i5 in [i4..lengthOfList(t3)-1]:
                            if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i1],t3[i3])==1 and IsWeaklySeparated(t3[i1],t3[i4])==1 and IsWeaklySeparated(t3[i1],t3[i5])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1 and IsWeaklySeparated(t3[i2],t3[i4])==1         and IsWeaklySeparated(t3[i2],t3[i5])==1 and IsWeaklySeparated(t3[i3],t3[i4])==1 and IsWeaklySeparated(t3[i3],t3[i5])==1 and IsWeaklySeparated(t3[i4],t3[i5])==1:
                                r1=append(r1, [t3[i1],t3[i2],t3[i3],t3[i4],t3[i5]])
        
        r=r1


    elif N==6:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    for i4 in [i3..lengthOfList(t3)-1]:
                        for i5 in [i4..lengthOfList(t3)-1]:
                            for i6 in [i5..lengthOfList(t3)-1]:
                                if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i1],t3[i3])==1 and IsWeaklySeparated(t3[i1],t3[i4])==1 and IsWeaklySeparated(t3[i1],t3[i5])==1        and IsWeaklySeparated(t3[i1],t3[i6])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1         and IsWeaklySeparated(t3[i2],t3[i4])==1 and IsWeaklySeparated(t3[i2],t3[i5])==1         and IsWeaklySeparated(t3[i2],t3[i6])==1 and IsWeaklySeparated(t3[i3],t3[i4])==1 and IsWeaklySeparated(t3[i3],t3[i5])==1 and IsWeaklySeparated(t3[i3],t3[i6])==1         and IsWeaklySeparated(t3[i4],t3[i5])==1 and IsWeaklySeparated(t3[i4],t3[i6])==1 and IsWeaklySeparated(t3[i5],t3[i6])==1:
                                    r1=append(r1, [t3[i1],t3[i2],t3[i3],t3[i4],t3[i5],t3[i6]])
                                
        r=r1 
    return r 

def ExtendSetOfTableauxToContainEvacuationsInMatrix(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicatesListOfLists(r)
    
    return r

def ExtendSetOfTableauxToContainEvacuationsInMatrix2(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicates2(r)
    
    return r

def EvacuationOfPolynomialInPluckerNTimesListForPair(N,ll,n): # ll=[T,l], T is a list of lists, l = ch(T) = [[1,[[],[]]], [-1,[[],[]]]]  Gr(k,n), collect Evacuations 0 time, 1 time, 2 times, ..., N-1 times
    T=ll[0]
    l=ll[1]
    k=len(T[0])
    
    M=RandomTotallyNonNegativeG(k,n,2,633)
    t1=ll[1]
    g1=PluckerToMinorsPolynomial(M, t1)
    if g1<0:
        #print('t1',t1)
        t1=Multiplyq2(t1,-1)

    r=[[ll[0], t1]]
    
    for i in [1..N-1]:
        #print(T)
        t0=EvacuationOfTableauNTimesTheNthTisListOfLists(i,T,n)
        #print(t0)
        t1=EvacuationOfPolynomialInPluckerNTimes(i,l,n)
        
        
        M=RandomTotallyNonNegativeG(k,n,2,633)
        g1=PluckerToMinorsPolynomial(M, t1)
        if g1<0:
            #print('t1',t1)
            t1=Multiplyq2(t1,-1)
            #print('t1',t1)
        
        r.append([t0,t1])
    #r=removeDuplicatesListOfLists(r) # sometimes has problem when removing duplicates
    return r
    
def EvacuationOfPolynomialInPluckerNTimes(N,l,n): # Gr(k,n), add N to indices and sort, terms in l are noncrossing does not imply that the resulting polynomial has noncrossing terms
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=[]
        for j in t2:
            #print('j',j,TableauToMatrix([j]))
            t4=EvacuationOfTableauNTimesInMatrix(N+1,TableauToMatrix([j]),n)[-1]
            t3.append(MatrixTakeColumns(t4)[0]) 
        r.append([t1, t3]) 
    return r
    
def EvacuationOfTableauNTimesTheNthTisListOfLists(N,T1,n): # T1 is a list of lists
    r2=TableauToMatrix(T1)
    r3=SemistandardTableau(MatrixTakeRows(r2))
    r4=EvacuationOfTableauNTimesTheNth(N,r3,n)
    r5=TableauToMatrixTakeRows(r4)
    r=MatrixTakeColumns(r5)  
    return r


def EvacuationOfTableauNTimesTheNth(N,T1,n): # T1 is a semistandard tableau
    T=T1
    for i in range(N):
        T=T.evacuation(n)
    r=T
    return r
    
def EvacuationOfTableauNTimes(N,T1,n):
    r=[T1]
    T=T1
    for i in range(N-1):
        T=T.evacuation(n)
        #print('T',T)
        r.append(T)
    return r 

def EvacuationOfTableauNTimesInMatrix(N,T1,n):
    t1=MatrixTakeRows(T1)
    t2=SemistandardTableau(t1)
    #print(t2)
    r1=EvacuationOfTableauNTimes(N,t2,n)
    r=[]
    for i in r1:
        r.append(TableauToMatrixTakeRows(i))
        
    return r 

def TableauxToListOfTimesOfOccurrenceOfNumbersNEntries(n,a): # Gr(k,n)
    r=[]
    k=a.nrows() 
    m=a.ncols() 

    r1=[]
    for i in [0..k-1]:  
        for j in [0..m-1]: 
            r1=append(r1, a[i,j]) 

    for j in [1..n]:
        t1=0
        for i in r1:
            if i==j:
                t1=t1+1 
        r=append(r,t1)


    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsListNEntries(n,a):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbersNEntries(n,t1)
 
    return r

def TakeRowIOfMatrix(i, l):
    r=[]
    for j in [0..l.ncols()-1]:
        r.append(l[i-1,j])
    return r


def lengthOfList(l):
    r=len(l)
    return r

def UnionOfLists(a,b):
    r=a+b
    return r
def append(l,i):
    r=l
    r.append(i)
    return r

def TableauToFundamentalTableauxSLkOneColumn(l1): # l1 is a tableau with one column, l1 is in list form
 
    r=[] 
    l=sorted(l1)
    for i in [0..len(l)-1]:
        if l[i]!=i+1 and l[i]<10000:
            t1=ListAToN(1,i)
            t1=append(t1,l[i])
            r=append(r,t1)
      

    return(r) 

def ReverseList(l):
#     r=[]
#     for i in range(len(l)-1,-1,-1):
#         r=r+[l[i]]
    r=l[::-1]
    
    return r


def TableauToFundamentalTableauxSLk(Y,l): # l is a tableau T, l[i] is column of i of T
 

    r=[]

    for i in [0..len(l)-1]: 
        r=UnionOfLists(r, TableauToFundamentalTableauxSLkOneColumn(l[i]))
    r=sorted(r)
    #print(r)
    r=ReverseList(r)

    return(r) 


def TableauToSmallGapTableauSeparated(a): # a is tableau written in terms of a matrix
    a1=MatrixTakeColumns(a)
    #print(a1)
    r1=TableauToSmallGapTableau(6,a1) # TableauToSmallGapTableau(k,a) does not depend on k
    r=[]
    #print(r1)
    for i in r1:
        r.append(TableauToMatrix([i]))
    return r
        
def PABC(a,b,c,n):
    r=[]
    for i in range(n):
        r.append(0)
    r[0]=b
    for i in range(1,a):
        r[i]=r[i-1]+1
    r[a]=r[a-1]+c
    
    for i in range(a+1,n):
        r[i]=r[i-1]+1
    return r

def CollectJumpPointsForPlucker(a1):
    a=sorted(a1)
    n=len(a)
    r=[]
    for i in range(n-1):
        if a[i+1]-a[i]>1:
            r.append(i)
    return r

def PluckerToKR(a1):
    a=sorted(a1)
    n=len(a)
    t2=CollectJumpPointsForPlucker(a);
    #print(t2)
    r=[]
    for i in t2:
        i1=i+1
        t1=PABC(i1, a[i]-i1+1, a[i+1]-a[i],n);
        r.append(t1)
    return r

def FundamentalModulesToPlucker(a1, n): # a1 is the index of Y_{i,s}, n is the number of rows of the tableau
    i=a1[0];
    s=a1[1];

    r=PABC(n-i, (i-s)/2, 2, n);
    return r

def KRToFundamentalModules(a1):
    a=sorted(a1)
    n=len(a)
    t2=CollectJumpPointsForPlucker(a);
    k=t2[0]
    r=[]
    for i in range(a[k]+1, a[k+1]):
        k1=k+1
        i1=i+1
        t1=PABC(k1, a[0]+i1-a[k]-2, 2,n);
        r.append(t1)
    return r




def ModulesToSmallGapTableau(a1, n): # a1 is the list of indices of a module Y_{i1,s1}...Y_{ir,sr} n is the number of rows of tableau 
    r=[]
    for i in a1:
        t1=[FundamentalModulesToPlucker(i, n)]
        print(t1)
        r=UnionOfTwoTableaux(r, t1);
        
    return r

def ModulesToTableau(a1, n,m): # a1 is the list of indices of a module Y_{i1,s1}...Y_{ir,sr} n is the number of rows of tableau 
# m can take sufficient large
    t1=a1
    r=[]
    for i in t1:
        r=UnionOfTwoTableaux(r, [FundamentalModulesToPlucker(i, n)]);
    
    t2=PluckersWhichAreOne(n,m);
    sn=1
    while sn!=0:
        r1=r
        for i in range(len(t2)):
            if IsTableauAFactorOfB([t2[i]], r)==1:
                r=TableauDivision(r, [t2[i]]);
        if r1==r:
            sn=0
            break
        else:
            sn=1
    return r

def PluckerToFundamentalModules(a1):
    t1=PluckerToKR(a1);
    r=[]
    for i in t1:
        t2=KRToFundamentalModules(i);
        r=r+t2
    return r

def TableauToSmallGapTableau(k,a): # a is a list, a[i]'s are columns of a tableau
    t1=[]
    for i in a:
        t2=PluckerToFundamentalModules(i)
        #print(t2)
        t1=t1+t2
    r=sorted(t1)
    return r

def CollectFirstElementInSmallGapTableau(a1):
    r=[]
    for a in a1:
        a2=sorted(a)
        r.append(a2[0])
    return r

def CollectMissingElementInSmallGapTableau(a1):
    r=[]
    for a2 in a1:
        a=sorted(a2)
        n=len(a)
        #print(a)
        #print(a)
        for i in range(n-1):
            if a[i+1]-a[i]>1:
                r.append((a[i]+a[i+1])/2)
                break
    return r 

def HighestWeightTableauInPolynomialOfTableaux(l,typ,rank): # l is a list, l[i]=[coeff, tableau], rank is k-1 in A_{k-1}, Gr(k,n)
    if l==[]:
        r=[]
    else:
        r=l[0]
        for i in [1..len(l)-1]:
            if compareWeightsTableaux2(r[1], l[i][1], typ,rank)==-1:
                
                #print('r[1]',TableauToModuleHasY(Y,r[1]))
                #print('l[i][1]',TableauToModuleHasY(Y,l[i][1]))
                r=l[i]
                
    return r

def compareWeightsTableaux2(P1,P2,typ,rank): # a,b are tableaux, rank is k-1 in A_{k-1}, Gr(k,n)
    if P1!=[]:
        t1=WeightOfTableau2(P1)
    else:
        t1=[]
        for i in [1..rank]:
            t1.append(0)

    if P2!=[]:
        t2=WeightOfTableau2(P2)
    else:
        t2=[]
        for i in [1..rank]:
            t2.append(0)

    #print(t1,t2)

    r=compareWeights2(t1,t2,typ, rank)

    return r

def WeightOfTableau2(a): # a[i] are columns of the tableau a
    #print('a', a)
    m=len(a)
    n=len(a[0])
    
    #print(m,n)
    
    r=[]
    for i in [1..n-1]:
        r.append(0)
    
    #print(r)
    
    for i in [0..m-1]:
        t1=PluckerToMinimalAff2(a[i])
        #print(t1)
        r=AddVlueOfTwoLists(r,t1) 
        
    return r

def PluckerToMinimalAff2(a1):
 
    r=[]
    a=sorted(a1)
    n=len(a)
    for i in range(n-1,0,-1):
        r.append(a[i]-a[i-1]-1)
     
    #print(r)

    return r

def AddVlueOfTwoLists(a,b):
    r=copy.deepcopy(a)
    for i in [0..len(a)-1]:
        r[i]=a[i]+b[i]
    return r

def compareWeights2(a,b,typ,rank): # a,b are lists
    n=len(a)
    t1=Matrix(n,1)
    for i in range(n): 
        t1[i,0]=a[i] 
    t2=Matrix(n,1)
    for i in range(n): 
        t2[i,0]=b[i] 
    r=compareWeights(t1,t2,typ,rank)

    return r


def TupleWhoseUnionIsT(T): # maybe there are other faster way, T is not necessarily semistandard
    """
    Iterator over the non crossing tuples with same content as T.
    """
    if not T or not T[0]:
        return []

    k = len(T[0])
    m = len(T)  # number of columns of T
    content = sorted(flatten(T))

    # Unfortunately, we need to record yielded solutions to avoid duplicates
    #seen = set()
    r=[]
    for c in candidates(Counter(content), k, m):
        #print(c)
        if SortRowsOfTableau(c)==SortRowsOfTableau(T):
            r.append(sorted(c)) 
    r=removeDuplicatesListOfLists(r)
    return r

def NonCrossingTupleWithSameContentAndWeightAsT(T): # maybe there are other faster way, T can be non semistandard
    r1=NonCrossingTupleWithSameContentAsT_new(T)
    for c in r1:
        if SortRowsOfTableau(c)==SortRowsOfTableau(T):
            r=c
            break
    return r

def PromotionOfPolynomialInPluckerNTimesListForPair(N,ll,n): # ll=[T,l], T is a list of lists, l = ch(T) = [[1,[[],[]]], [-1,[[],[]]]]  Gr(k,n), collect promotions 0 time, 1 time, 2 times, ..., N-1 times
    T=ll[0]
    l=ll[1]
    k=len(T[0])
    
    M=RandomTotallyNonNegativeG(k,n,2,633)
    t1=ll[1]
    g1=PluckerToMinorsPolynomial(M, t1)
    if g1<0:
        #print('t1',t1)
        t1=Multiplyq2(t1,-1)

    r=[[ll[0], t1]]
     
    for i in [1..N-1]:
        t0=PromotionOfTableauNTimesTheNthTisListOfLists(i,T,n)
        t1=PromotionOfPolynomialInPluckerNTimes(i,l,n)
        
        t2=t1
        M=RandomTotallyNonNegativeG(k,n,2,633)
        g1=PluckerToMinorsPolynomial(M, t1)
        if g1<0:
            #print('t1',t1)
            t2=Multiplyq2(t1,-1)
            #print('t2',t2)
        
        r.append([t0,t2])
    #r=removeDuplicatesListOfLists(r) # sometimes has problem when removing duplicates
    return r

def PluckerPolynomialTableauExpansionsOneExpansion(l,b,c): # l is [[1, [[1,3,7],[2,5,8]]], [1, [[1,5,7],[2,6,9]]]]
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=TableauExpansionsHalf(t2,b,c)
        r.append([t1, t3])    
    
    return r

# def PluckerPolynomialTableauExpansions(ll,n): # ll = [T, l], l is ch(T) = [[1, [[1,3,7],[2,5,8]]], [1, [[1,5,7],[2,6,9]]]]
#     T=ll[0]
#     l=ll[1]
#     r1=ContentOfTableau(T)
#     m=len(r1)
#     r2=list(itertools.combinations(list(range(1,n+1)), m)) 
#     r=[]
#     for i in r2:
#         t0=TableauExpansionsHalf(T,r1,i)
#         t1=PluckerPolynomialTableauExpansionsOneExpansion(l,r1,i)
#         r.append([t0, t1])
#     return r

def PluckerPolynomialTableauExpansions(ll,n): # ll = [T, l], l is ch(T) = [[1, [[1,3,7],[2,5,8]]], [1, [[1,5,7],[2,6,9]]]]
    
    T=ll[0]
    l=ll[1]
    k=len(T[0])
    
    r1=ContentOfTableau(T)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t0=TableauExpansionsHalf(T,r1,i)
        t1=PluckerPolynomialTableauExpansionsOneExpansion(l,r1,i)
        
        M=RandomTotallyNonNegativeG(k,n,2,633)
        g1=PluckerToMinorsPolynomial(M, t1)
        if g1<0:
            #print('t1',t1)
            t1=Multiplyq2(t1,-1)
            #print('t1',t1)
        
        r.append([t0, t1])
    return r

def TableauExpansionsHalf(l,b,c): # l[i] is ith column of the tableau
    r=[]
    m=len(l[0])
    n=len(l)
    r=[]
    
    #print(b,c)
    
    for i in range(n):
        t1=[]
        for j in range(m):
            t2=b.index(l[i][j])
            t1.append(c[t2])
        r.append(t1)

    return r

def TableauExpansions(l,n): # l[i] is a column
    r1=ContentOfTableau(l)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t1=TableauExpansionsHalf(l,r1,i)
        r.append(t1)
    return r

def TableauExpansionsList(l,n): # l is a list of tableaux in list form
    r=[]
    for i in l:
        r=r+TableauExpansions(i,n)
    r=removeDuplicatesListOfLists(r)  
    
    return r

def NumberOfInversions(arr):
    inv_count = 0
    n=len(arr)
    for i in range(n):
        for j in range(i + 1, n):
            if (arr[i] > arr[j]):
                inv_count += 1
  
    return inv_count

def PromotionOfPolynomialInPlucker(l,n): # Gr(k,n), add 1 to indices and sort
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=[]
        for j in t2:
            t4=ShiftEveryIndexByNMondMList(j,1,n)
            t5=sorted(t4)
            t1=t1*(-1)^NumberOfInversions(t4)
            #print(t1)
            t3.append(t5)
        r.append([t1, t3]) 
    return r

def PromotionOfPolynomialInPluckerNTimes(N,l,n): # Gr(k,n), add N to indices and sort, terms in l are noncrossing does not imply that the resulting polynomial has noncrossing terms
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=[]
#         for j in t2:
#             t4=ShiftEveryIndexByNMondMList(j,N,n)
#             t5=sorted(t4)
#             t1=t1*(-1)^NumberOfInversions(t4)
#             #print(t1)
#             t3.append(t5)

        for j in t2:
            t4=PromotionOfTableauNTimesInMatrix(N+1,TableauToMatrix([j]),n)[-1]
            t3.append(MatrixTakeColumns(t4)[0]) 
            
        r.append([t1, t3]) 
    return r
    
def PromotionOfPolynomialInPluckerNTimesList(N,l,n): # Gr(k,n), collect promotions 0 time, 1 time, 2 times, ..., N-1 times
    r=[l]
    for i in [1..N-1]:
        t1=PromotionOfPolynomialInPluckerNTimes(i,l,n)
        r.append(t1)
    
    return r

def ShiftEveryIndexByNMondMList(l,n,m):
    r=[]
    for i in l:
        t1=ModResultOneToN(i+n, m)
        r.append(t1)
    return r

def ShiftEveryIndexByNMondMListOfLists(l,n,m):
    r=[]
    for i in l:
        t1=ShiftEveryIndexByNMondMList(i,n,m)
        r.append(t1)
    return r

def ShiftEveryIndexInPluckerPolynomialByNModM(l, n,m):
    r=[]
    for i in l:
        t1=ShiftEveryIndexByNMondMListOfLists(i[1],n,m)
        r.append([i[0], t1])
    return r

def ModResultOneToN(k,n):
    if k%n!=0:
        r=k%n
    else:
        r=n
    return r

def ChooseTableauxWithSmallOrEqualWeight(T, l,typ,rank):
    r=[]
    for i in l:
        if compareWeightsTableaux(T, i, typ, rank)==1: # weight of T is larger or equal to weight of i
            r.append(i)
    return r

def merge_two_dicts(d1, d2):  
    d={}
    d2_keys_not_in_d1 = d2.keys() - d1.keys()
    d1_keys_not_in_d2 = d1.keys() - d2.keys()
    common_keys = d2.keys() & d1.keys()

    for i in common_keys:
        d[i]= removeDuplicates( d1[i]+d2[i] )
    for i in d1_keys_not_in_d2:
        d[i]=d1[i]
    for i in d2_keys_not_in_d1:
        d[i]=d2[i]
    return d
  
def merge_two_dicts_update(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

def detSelfDefined(M):
    M = [row[:] for row in M] # make a copy to keep original M unmodified
    N, sign, prev = len(M), 1, 1
    for i in range(N-1):
        if M[i][i] == 0: # swap with another row having nonzero i's elem
            swapto = next( (j for j in range(i+1,N) if M[j][i] != 0), None )
            if swapto is None:
                return 0 # all M[*][i] are zero => zero determinant
            M[i], M[swapto], sign = M[swapto], M[i], -sign
        for j in range(i+1,N):
            for k in range(i+1,N):
                assert ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) % prev == 0
                M[j][k] = ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) // prev
        prev = M[i][i]
    return sign * M[-1][-1]

def UnionOfTableauxMany(a):
 
    r=0

    if a!=[]:
        r=a[0]
        for i in [1..len(a)-1]:
            r=UnionOfTwoTableaux(r, a[i])
         
    return r

# def NonCrossingTupleWithSameContentAsT(T): # T = [[1,2,3,5,8],[2,3,4,8,9]] is a tableau
#     r=[]
#     if T != []:
#         k=len(T[0])
#         m=len(T) # number of columns of T
#         n=0
#         for i in T:
#             if n<max(i):
#                 n=max(i)
    
#     for i in itertools.combinations_with_replacement(itertools.combinations(ListAToN(1,n),k), m):
#         if ContentOfTableauWithMultiplicities(i)==ContentOfTableauWithMultiplicities(T) and IsNonCrossingList(i)==1:
#             r.append(i)
#     #r=removeDuplicatesListOfLists(r) 
    
#     return r

# def NonCrossingTupleWithSameContentAsT(T):
#     """
#     Iterator over the non crossing tuples with same content as T.
#     """
#     if not T or not T[0]:
#         return

#     k = len(T[0])
#     m = len(T)  # number of columns of T
#     content = sorted(flatten(T))

#     # Unfortunately, we need to record yielded solutions to avoid duplicates
#     seen = set()

#     for c in candidates(Counter(content), k, m):
#         if IsNonCrossingList(c):
#             t = tuple(sorted(c))
#             if t not in seen:
#                 yield t
#                 seen.add(t)

def NonCrossingTupleWithSameContentAsT(T):
    """
    Iterator over the non crossing tuples with same content as T.
    """
    if not T or not T[0]:
        return []

    k = len(T[0])
    m = len(T)  # number of columns of T
    content = sorted(flatten(T))

    # Unfortunately, we need to record yielded solutions to avoid duplicates
    #seen = set()
    r=[]
    for c in candidates(Counter(content), k, m):
        if IsNonCrossingList(c):
            r.append(tuple(sorted(c)))
#             if t not in seen:
#                 yield t
#                 seen.add(t)
    r=removeDuplicatesListOfLists(r)
    return r

# def IsNonCrossingList(l): # I1,J1 have the same size
    
#     r=1
#     if len(l)>=2:
#         for i in itertools.combinations(l, 2):
#             if IsNonCrossing(i[0],i[1])==0:
#                 r=0
#                 return r
    
#     return r 

def IsNotWeaklySeparatedForAnyPairInList(L): # any pair is not weakly separated
    r=0
    if len(L)>=2:
        r=1
        for i in itertools.combinations(L, 2):
            if IsWeaklySeparated(i[0],i[1])==1:
                r=0
                return r
    return r

def AtLeastOnePairIsNotWeaklySeparatedInList(L): # at least one pair is not weakly separated
    r=0
    if len(L)>=2:
        r=0
        for i in itertools.combinations(L, 2):
            if IsWeaklySeparated(i[0],i[1])==0:
                r=1
                return r
    return r

def IsNonCrossingList(L):
    return all(IsNonCrossing(I, J) for I, J in itertools.combinations(L, 2))

def candidates(content, k, m):
    """
    Iterator over the m-tuples of k elements from content.

    With this implementation, elements are not repeated in a same tuple.

    INPUT:

    - ``content`` -- a ``Counter`` associated to each possible element it's
      multiplicity

    - ``k`` -- number of elements per tuple

    - ``m`` -- requested number of tuples
    """
    if len(content) < k:
        return
    for I in itertools.combinations(content, k):
        # We ensure that elements inside a k-tuple are sorted
        T = (tuple(sorted(I)), )
        if m == 1:
            yield T
        else:
            for L in candidates(content - Counter(I), k, m - 1):
                yield T + L

def MatrixTakeColumns(a):
    n=a.nrows()
    m=a.ncols()
    r=[]
    for j in range(m):
        t1=a[list(range(n)), [j]]
        t2=[]
        for i in range(n):
            t2.append(t1[i,0])
        r.append(t2)
    return r

# def ComputeChTInTermsOfNonCrossingBasis(l): # noncrossing basis, NonCrossingTupleList is l[0], l[1] is list of pairs (M, value of ch(T) on M)
  
#     c=VariableGenerator('c') 
#     g1=l[0]

#     var=[]
#     for j in [0..len(g1)-1]:
#         var.append(c[j])
    
#     p1=[]
#     for j1 in range(len(l[0])):
#         j=l[1][j1]
#         M=j[0]
#         r3=j[1]
#         r4=0 
#         for p in [0..len(g1)-1]:
#             #print(g1[p])
#             r4=r4+PluckerToMinorsPolynomial(M, [[c[p], g1[p]]]) 
#             #print(r4)
#         #print('r3',r3)
#         #print('r4',r4)
#         p1.append(r3-r4)
#     #print(p1)
#     t1=solve(p1, var, solution_dict=True)
#     #print('t1',t1)
#     t5=[]
#     #t5.append([1, i[0]])
#     for q in [0..len(g1)-1]:
#         t5.append([round(t1[0][c[q]]), g1[q]])
#     t5=RemoveZeroTerms(t5)
#     r=t5
    
#     return r

def ComputeChTInTermsOfNonCrossingBasis(l): # noncrossing basis, NonCrossingTupleList is l[0], l[1] is list of pairs (M, value of ch(T) on M)

    g1=l[0]
    n=len(l[0])
    p1=np.zeros([n,n])
    p2=np.zeros([n])
    for j1 in range(n):
        j=l[1][j1]
        M=j[0] 
        #print('j',j)
        p2[j1]=j[1] 
        for p in [0..n-1]:
            #print(g1[p])
            p1[j1, p]=PluckerToMinors(M, [1, g1[p]]) 
         
    #t1=np.linalg.solve( p1,p2 )
    #t1=scipy.linalg.inv(p1).dot(p2)
    t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #print('t1',t1)
    t5=[]
    #t5.append([1, i[0]])
    for q in [0..len(g1)-1]:
        t5.append([int(round(t1[q])), g1[q]])
    r=RemoveZeroTerms(t5) 
    
    return r
def ComputeChTInTermsOfNonCrossingBasisUpToRound100th(l): # noncrossing basis, NonCrossingTupleList is l[0], l[1] is list of pairs (M, value of ch(T) on M)

    g1=l[0]
    n=len(l[0])
    p1=np.zeros([n,n])
    p2=np.zeros([n])
    for j1 in range(n):
        j=l[1][j1]
        M=j[0] 
        #print('j',j)
        p2[j1]=j[1] 
        for p in [0..n-1]:
            #print(g1[p])
            p1[j1, p]=PluckerToMinors(M, [1, g1[p]]) 
         
    #t1=np.linalg.solve( p1,p2 )
    #print('t1',t1)
    #t1=scipy.linalg.inv(p1).dot(p2)
    t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=scipy.linalg.lstsq(p1, p2)[0] # ??? check
    t5=[]
    #t5.append([1, i[0]])
    for q in [0..len(g1)-1]:
        t5.append([round(t1[q],3), g1[q]])
    r=RemoveZeroTerms(t5) 
    
    return r


class VariableGenerator(object): 
     def __init__(self, prefix): 
         self.__prefix = prefix 
     @cached_method 
     def __getitem__(self, key): 
         return SR.var("%s%s"%(self.__prefix,key)) 

def NchooseM(s, m):
    r=list(itertools.combinations(s, m))
    return r

def ContentOfTableauWithMultiplicities(l): # l is tableau
    r=[]
    for i in l:
        for j in i:
            r.append(j)
    r=sorted(r)
    return r

# def IsNonCrossing(I1,J1): # I1,J1 have the same size
#     k=len(I1)
#     I2=sorted(I1)
#     J2=sorted(J1)
#     r=1
#     for a in [0..k-1]:
#         for b in [a+1..k-1]:
#             t1=[]
#             for i in [a..b]:
#                 t1.append(I2[i])
#             t2=[]
#             for j in [a..b]:
#                 t2.append(J2[j])
#             t3=[]
#             for i in [a+1..b-1]:
#                 t3.append(I2[i])
#             t4=[]
#             for j in [a+1..b-1]:
#                 t4.append(J2[j])
             
            
#             if IsWeaklySeparated(t1,t2)==0 and set(t3)==set(t4):
#                 return(0)
#     return r 

def IsNonCrossing(I,J): # I,J have the same size
    k = len(I)

    for a in range(k - 1):
        for b in range(a + 1, k):
            if (I[a+1:b] == J[a+1:b] and
                    not IsWeaklySeparated(I[a:b+1], J[a:b+1])):
                return False

    return True

def IsIncreasingOrdered(l): # is there j such that l_j<...<l_n<l_1<...<l_{j-1}
 

    r=1
    n=len(l)
    for j in [0..n-2]:
        if l[j]>l[j+1]:
            r=0
            break
         
    return r 


# def IsWeaklySeparated(I1,J1): # I1,J1 have the same size
 

#     r1=sorted(SetDifference(I1,J1))
#     r2=sorted(SetDifference(J1,I1))

#     if r1==[] and r2==[]:
#         r=1
#     else:
#         #print(r1,r2)
#         if min(r1)<min(r2):
#             c1=r1
#             c2=r2
#         else: 
#             c1=r2
#             c2=r1 
            
#         r=0

#         for i in [0..len(c1)-1]:
#             t1=[]
#             for j in [0..i]:
#                 t1.append(c1[j])
                
#             t1=t1+c2
#             for j in [i+1..len(c1)-1]:
#                 t1.append(c1[j])
                 
#             if IsIncreasingOrdered(t1)==1:
#                 r=1
#                 break
             
#     return r 

def IsWeaklySeparated(I,J): # I,J have the same size
#     r1 = sorted(set(I).difference(J))
#     r2 = sorted(set(J).difference(I))
    r1=sorted(SetDifference(I,J))
    r2=sorted(SetDifference(J,I))

    if not r1 or not r2:
        return True

    if min(r1) >= min(r2):
        r1, r2 = r2, r1

    # Search for an index i such that list r1[:i+1] + r2 + r1[i+1:] is sorted
    for i in range(len(r1) - 1):
        if r2[0] < r1[i+1]:
            # We check if list r1[:i+1] + r2 + r1[i+1:] is ordered
            return r2[-1] <= r1[i+1]

    # It remains to check if list r1 + r2 is ordered
    return r1[-1] <= r2[0]
 
def QaRoots(k,a):
    r1=0
    for i in a:
        r1=r1+i^2
    r2=0
    for i in a:
        r2=r2+i
    r=r1+(2-k)/(k^2)*r2^2

    return r


def RemoveZeroTerms(l):
 
    r=[]
    for i in l:
        if (i[0]!=0 and i[1]!=0):
            r.append(i)
        
    return r

def QaRootsOfTableaux(k,l): # l is a tableaux written as a matrix
    r1=TableauxToListOfTimesOfOccurrenceOfNumbers(l)
    r=QaRoots(k,r1)

    return r

def TableauToMatrixTakeRows(a): # a[i] are rows
    m=len(a)
    n=len(a[0])
    r=Matrix(m,n)
    for i in range(m):
        for j in range(n):
            r[i,j]=a[i][j]
    return r

def PromotionOfTableauNTimesTheNth(N,T1,n): # T1 is a semistandard tableau
    T=T1
    for i in range(N):
        T=T.promotion(n-1)
    r=T
    return r

def PromotionOfTableauNTimesTheNthTisListOfLists(N,T1,n): # T1 is a list of lists
    r2=TableauToMatrix(T1)
    r3=SemistandardTableau(MatrixTakeRows(r2))
    r4=PromotionOfTableauNTimesTheNth(N,r3,n)
    r5=TableauToMatrixTakeRows(r4)
    r=MatrixTakeColumns(r5)  
    return r

def PromotionOfTableauNTimes(N,T1,n):
    r=[T1]
    T=T1
    for i in range(N-1):
        T=T.promotion(n-1)
        r.append(T)
    return r 

def PromotionOfTableauNTimesInMatrix(N,T1,n):
    t1=MatrixTakeRows(T1)
    t2=SemistandardTableau(t1)
    r1=PromotionOfTableauNTimes(N,t2,n)
    r=[]
    for i in r1:
        r.append(TableauToMatrixTakeRows(i))
        
    return r 

def PluckerToMinimalAff(a1):
    r=[]
    a=sorted(a1)
    n=len(a)
    for i in range(n-1,0,-1):
        r.append(a[i]-a[i-1]-1)
    r.append(a[0]-1)

    return r

def PluckerToMinors(A,l):
    r=l[0]
    for i in l[1]:
        r=r*Minor(A, ListAToN(1, len(i)), i) 
    return r

def PluckerToMinorsPolynomial(A,l):
    r=0
    for i in l:
        r=r+PluckerToMinors(A,i) 
    r=simplify(r)
    return r



def subMatrix(m, rows, cols):
    result = []
    for row in rows:
        newRow = []
        for col in cols:
            newRow.append(m[row-1][col-1])
        result.append(newRow)
    #print(result)
    r=Matrix(np.random.rand( len(rows), len(cols) ))
    for i in range(len(rows)):
        for j in range(len(cols)):
            r[i,j]=result[i][j]
    
    return r



def Minor(M, rows, cols):
    #r=np.linalg.det(subMatrix(M,rows, cols))
    #r=det(subMatrix(M,rows, cols))
    r=det(sub_matrix_general(M,rows, cols))
    #r=detSelfDefined(subMatrix(M,rows, cols))
    return r

def InitialCluster(rank,n,MatrixGList): # Gr(rank,n)
    
    sizeColumn=n-rank
    k=sizeColumn    
    k1=rank
    p1=k*rank+1
    mat=Matrix(p1,p1)
    for i in range(p1): 
        i1=i+1
        if i1==1: 
            mat[i,i+k+1]=1
            mat[i, p1-1]=1
            mat[i,i+k]=-1
            mat[i, i+1]=-1
        elif i1>=2 and i1<=k-1: 
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1==k: 
            mat[i,i-1]=1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+k+1]=1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k >=2 and i1 % k<=k-1:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1>=2*k and i1<=(rank-1)*k and i1 % k==0:
            mat[i,i-k-1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1>(rank-1)*k and i1<p1 and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+1]=-1
        elif i1>=(rank-1)*k+2 and i1<rank*k:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==rank*k:
            mat[i,i-k-1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==p1:
            mat[i,0]=-1

    vertices0=[]

    for j in range(k1-1,-1,-1):
        for i in range(k1,k1+k):
            t1=list(range(1,j+1))
            t2=list(range(i-k1+j+2,i+2))
            t3=t1+t2
            vertices0.append(t3)

    vertices0.append(list(range(1,k1+1)))  
    verticesTableaux = [] # Tableaux are represented by matrices
    for i in range(len(vertices0)):
        #print('PluckerToMinors(MatrixG, [vertices0[i]])',PluckerToMinors(MatrixG, [vertices0[i]]))
        t1=[]
        for MatrixG in MatrixGList:
            t1.append( PluckerToMinorsPolynomial(MatrixG, [[1,[vertices0[i]]]]) )
        verticesTableaux.append([0, [vertices0[i]], t1, i]) # [vertices0[i]] is an one column tableau
    mat1 = Matrix(p1,p1)
    for i in range(p1):
        for j in range(p1):
            mat1[i,j]=mat[i,j]
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r


def TableauExpansionsInMatrixHalf(l,b,c): # l is tableau in matrix form, b is the content of l, c is a list of numbers
    r=[]
    m=l.nrows()
    n=l.ncols()
    r=Matrix(m,n)
    for i in range(m):
        for j in range(n):
            t1=b.index(l[i,j])+1
            r[i,j]=c[t1-1]
    return r

def TableauExpansionsInMatrix(l,n): # l is tableau in matrix form
    r1=ContentOfTableau(l)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t1=TableauExpansionsInMatrixHalf(l,r1,i)
        r.append(t1)
    return r

def TableauExpansionsInMatrixList(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        r=r+TableauExpansionsInMatrix(i,n)
    r=removeDuplicatesListOfLists(r)  
    
    return r

def TableauExpansionsInMatrixList2(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        r=r+TableauExpansionsInMatrix(i,n)
    r=removeDuplicates2(r)  
    
    return r


def ContentOfTableau(l): # l is tableau
    r=[]
    for i in l:
        for j in i:
            r.append(j)
    #r=np.unique(r,axis=0)
    r=removeDuplicatesListOfLists(r)
    r=sorted(r)
    return r

def immutabilize(m):
    M = copy.deepcopy(m)
    M.set_immutable()
    return M

def ChangeListOfMatricesToSetOfMatrices(S):
    r={immutabilize(i) for i in S}
    return r

def removeAnElementInList(i, l):
    r=[]
    for j in range(len(l)):
        if (j!=i):
            r.append(l[j])
    
    return r

def removeDuplicates(l):
    r=[]
    for i in l:
        if (i in r)==False:
            r.append(i)
    return r

def removeDuplicates2(l): # vary fast, for matrices
    t1=ChangeListOfMatricesToSetOfMatrices(l)
    r=list(dict.fromkeys(t1))
    return r

def removeDuplicatesListOfLists(l): # very fast
    l.sort()
    r=list(l for l,_ in itertools.groupby(l))

    return r


def SetDifference(A,B): # A-B
    r=[d for d in A if d not in B]
    return r 

def SetDifferenceListOfLists(A,B): # A-B, there are other faster ways
    
    r=[d for d in A if d not in B]
    
    return r

def SetDifference2(a,b):
    t1=ChangeListOfMatricesToSetOfMatrices(a)
    t2=ChangeListOfMatricesToSetOfMatrices(b)
    r=t1.difference(t2)
    return r

def SetDifferenceListDifference2(A,B): # A-B, can have duplicate elements
    r=[]
    r1=removeDuplicatesListOfLists(A)
    for i in r1:
        t1=A.count(i)-B.count(i)
        #print(t1)
        for j in range(1,t1+1):
            r.append(i)
            
    return r


def SetDifferenceListDifference(a, b):
    '''
    Return a copy of a with the elements of b removed.

    >>> subtract_multiset([], [0, 1])
    []
    >>> subtract_multiset(["x", "x", "y"], [])
    ['x', 'x', 'y']
    >>> subtract_multiset(["ham", "spam", "eggs", "spam", "spam"], ["spam", "spam"])
    ['ham', 'spam', 'eggs']
    >>> subtract_multiset([0, 0, 1, 1], [0, 1, 1, 1])
    [0]
    >>> subtract_multiset([1, 2, 2, 3], [1, 2, 3, 4])
    [2]
    '''
    diff = Counter(a) - Counter(b)
    return list(diff.elements())


def TableauToMatrix(a):
    m=len(a)
    n=len(a[0])
    r=Matrix(n,m)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r

def TableauToMatrixFloat(a):
    m=len(a)
    n=len(a[0])
    r=np.zeros((n,m))
    #print(r)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r

def TableauToMatrixList(a):
    r=[]
    for i in a:
        r.append(TableauToMatrix(i))
    return r


def MatrixTakeRows(a):
    n=a.nrows()
    m=a.ncols()
    r=[]
    for i in range(n):
        t1=a[[i],list(range(m))]
        t2=[]
        for j in range(m):
            t2.append(t1[0,j])
        r.append(t2)
    return r

def MatrixTakeRowsList(a):
    r=[]
    for i in a:
        r.append(MatrixTakeRows(i))
    return r

def MatrixTakeColumnsList(a):
    r=[]
    for i in a:
        r.append(MatrixTakeColumns(i))
    return r

def TableauDivision(a,b):
#     t1=TableauToMatrix(a)
#     t2=TableauToMatrix(b)  
#     r1=MatrixTakeRows(t1)
#     r2=MatrixTakeRows(t2)
    
    r1=TransposeTableauSLk(a)
    r2=TransposeTableauSLk(b)
    
#    print(r1,r2)
    
    r3=[]
    for i in range(len(r1)):
        t1=sorted(SetDifferenceListDifference(r1[i],r2[i]))
        if len(t1)!=len(r1[i])-len(r2[i]):
            print('not divisible')
            break
        else:
            r3.append(t1)
        #print(r1[i],r2[i])
    r=[]
    for i in range(len(r3[0])):
        t1=[]
        for j in range(len(r3)):
            t1.append(r3[j][i])
        r.append(t1)
        
    return r


def SortRowsOfTableau(l): # l[i] are columns of tableau
    r=[]
    for i in range(len(l[0])):
        r1=[]
        for j in range(len(l)):
            r1.append(l[j][i])
        r.append(sorted(r1))
    r2=r
    r=[]
    for i in range(len(r2[0])):
        r1=[]
        for j in range(len(r2)):
            r1.append(r2[j][i])
        r.append(r1)
    
    return r

def UnionOfTwoTableaux(a,b):
    t1=a+b
    r=SortRowsOfTableau(t1)
    return r


# def UnionOfTwoTableaux(a,b):
#     t1=a+b
#     t2=TableauToMatrix(t1)
#     r=[]
#     for i in range(t2.nrows()):
#         r1=[]
#         for j in range(t2.ncols()):
#             r1.append(t2[i,j])
#         r.append(sorted(r1))
        
#     r2=TableauToMatrix(r);
#     r=[]
#     for i in range(r2.nrows()):
#         r1=[]
#         for j in range(r2.ncols()):
#             r1.append(r2[i,j])
#         r.append(sorted(r1))
#     return r


        
# def PowerOfTableaux(a,n):
#     r=[]
#     if a!=[] and a!=[[]]:
#         for i in range(1,n+1):
#             r=UnionOfTwoTableaux(r,a) 
#     else:
#         r=a 
#     return r

        
def PowerOfTableaux(a,n):
    r=[]
    if a!=[] and a!=[[]]:
        for i in range(n):
            r=r+a
        r=SortRowsOfTableau(r) 
    else:
        r=a 
    return r



def compareWeightsTableaux(P1,P2,typ,rank): # a,b are tableaux
    t1=WeightOfTableau(P1)
    t2=WeightOfTableau(P2)
    r=compareWeights2(t1,t2,typ, rank)

    return r

def WeightOfTableau(a): # a[i] are columns of the tableau a
    m=len(a)
    n=len(a[0])
    r=[]
    for i in range(1,n+1):
        r.append(0)
    for i in range(m):
        t1=PluckerToMinimalAff(a[i])
        #r=list(np.array(r)+np.array(t1))
        r=list(map(add, r, t1))
        
    return r

def compareWeights(a, b, typ, rank): # 
    r=1                             # r=1 means a>=b          
    l=a-b
    c=CartanMatrixSelfDefined(typ, rank)
    for i in range(rank): 
        p=0
        for j in range(rank):
            t1=((transpose(c))^(-1))[j,i]
            p=p+l[j,0]*t1
        if p<0: 
            r=-1              # r=-1 means a is not >= b, it is possible that a<b or a,b are not comparable
            break 
            
    if r==-1:
        for i in range(rank):
            p=0
            for j in range(rank):
                t1=((transpose(c))^(-1))[j,i]
                p=p+l[j,0]*t1
            if p>0: 
                r=0
                break
    return r


def matrixMutation(mat,  k):  # mutate at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if k==i or k==j:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j]+1/2*(abs(mat[i,k])*mat[k,j]+mat[i,k]*abs(mat[k,j]))
     
    return r

def ExtendSetOfTableauxToContainPromotions(l,n): # l is a list of tableaux 
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimes(n,i,n)
        r=r+t1 
    #r=np.unique(r,axis=0)
    r=removeDuplicatesListOfLists(r)

    return r

def ExtendSetOfTableauxToContainPromotionsInMatrix(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicatesListOfLists(r)
    
    return r


def ExtendSetOfTableauxToContainPromotionsInMatrix2(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicates2(r)
    
    return r


def TableauxToListOfTimesOfOccurrenceOfNumbers(a):
        r=[]
        n=a.nrows() 
        m=a.ncols() 
        r1=[]
        for i in range(n):  
            for j in range(m): 
                r1.append(a[i,j]) 
        for k in range(1,max(r1)+1):
            t1=0
            for i in r1:
                if i==k:
                    t1=t1+1 
            r.append(t1)  
        return r

def TableauxToListOfTimesOfOccurrenceOfNumbersLengthN(a,N):
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j]) 
    for k in range(1,N):
        t1=0
        for i in r1:
            if i==k:
                t1=t1+1 
        r.append(t1)
    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersLengthNWithContentLessOrEquN(a,N): # compute the occurrences of numbers in i for those i in a such that the numbers in i is less or equal to N
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j])       
    if max(r1)<=N:
        for k in range(1,N):
            t1=0
            for i in r1:
                if i==k:
                    t1=t1+1 
            r.append(t1)
    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsList(a):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbers(t1)
 
    return r

def TableauxToListOfTimesOfOccurrenceOfNumbersLengthNTableauIsList(a,N):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbersLengthN(t1,N)
 
    return r

# mutation of Grassmannian cluster variables
def computeEquationsForModulesTableaux(variable2, mat, k, typ, rank): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
    
    #print('t1,newVariable,newVariable2,variable[k][1]',[t1,newVariableList,newVariable2List,variable[k][1]]) 

    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])  
        
    clusterVariables=TableauToMatrix(variable[k][1])
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def computeEquationsForModulesTableauxWithNumericalMutation(variable2, mat, k, typ, rank): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[] 

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) ) 
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) ) 
            
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
 
    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1]) 
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])   
        
    clusterVariables=TableauToMatrix(variable[k][1])
    
    #print('len(variable[0][2])',len(variable[0][2]))
    for j in range(len(variable[0][2])): # the jth matrix MatrixG
        #print('j',j)
        newVariableN=1
        newVariableN2=1
        for i in range(size):
            if mat[i, k]>0:
                newVariableN=newVariableN*variable[i][2][j]^mat[i,k]
                #print('newVariableN',newVariableN)
        for i in range(size): 
            if mat[i, k]<0:
                newVariableN2=newVariableN2*variable[i][2][j]^(-mat[i,k])
                #print('newVariableN2',newVariableN2)
        variable[k][2][j]=(newVariableN+newVariableN2)/variable[k][2][j]
        #print('j,variable[k][2][j]',j,variable[k][2][j])
    
    r=[variable, clusterVariables, variable[k][2]]

    return r


#Function for multiprocessing
def ll_perms(lli,typ,rank,max_column,n,repeat): 
    b1=[]
    IC=InitialCluster(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: ###why do this?, just to repeat the same sequence of mutations, it will give more cluster variables, without this loop is also ok
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            b1.append(vertices1[1]) 
            
#             if MatrixTakeRows(vertices1[1][len(vertices1[1])-1])==[[1,2,2,2],[3,4,4,4],[5,6,6,6],[7,8,8,8],[9,10,10,10]]:
#                 print(vertices1[1][len(vertices1[1])-1])

    b1=removeDuplicatesListOfLists(b1)
    b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
    b1=ExtendSetOfTableauxToContainEvacuationsInMatrix(b1,n)
    b1=TableauExpansionsInMatrixList(b1, n)
    
     

    return b1


# def ListAToN(a,n):
#     r=[]
#     for i in range(a,n+1):
#         r.append(i)
#     return r

def ListAToN(a,n):
    r=list(range(a,n+1))
    return r


def randomMatrix(rank,n,minn,maxn): # 

    r=np.random.randint(minn,maxn,(rank, n))
    r=Matrix(r)
    
    return r

def randomMatrixNonInt(rank,n): # rank times n matrix 

    r=np.random.rand(rank,n) 
    r=Matrix(r)

    return r

def RandomTotallyNonNegativeG(k,n, minn, maxn): # k<n

    L=sorted(random.sample(range(minn, maxn), max([k,n])))
    r=Matrix([[xi**i for i in range(len(L))] for xi in L]) 

    return r


def ll_permsWithNumericalMutation(lli,typ,rank,max_column,n,repeat,minn,maxn,NRandomM,CL1,CL2,fp1,fp2,b6,b_noncrossing): 


#def ll_permsWithNumericalMutation(lli,typ,rank,max_column,n,repeat,minn,maxn,NRandomM,CL1,CL2,fp1,b6,b_noncrossing): 
#    b6=b61.copy()
#    b6=b61
#    b_noncrossing=b_noncrossing1.copy()

    #MatrixG=np.random.randint(minn,maxn,(rank, n))
    MatrixGList=[]
    for i in range(NRandomM):
        #t1=RandomTotallyNonNegativeG(k,n,minn,maxn)
        #t1=randomMatrix(k,n,minn,maxn)
        t1=randomMatrixNonInt(k,n)
        MatrixGList.append(t1)
    
    #print('MatrixGList',MatrixGList)
    
    IC=InitialCluster(rank,n,MatrixGList)
    
    #print(IC[1])
    
    mat1=IC[0]
    vertices1=IC[1]
    
    #print('vertices1', vertices1)
    
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    F1 = open(fp1,'a+')
    F2 = open(fp2,'a+')
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        ii=MatrixTakeColumns(vertices1[1])
        p1=repr(ii)
        p2=b6.get(p1)

#         if vertices1[1].ncols()>max_column or p2==[]:
#             if p2==[]:
#                 print('ii is finished computing', ii)
#             vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank) 
#             mat1 = matrixMutation(mat1, mutationSequence[j])
            
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j])
        
        elif vertices1[1].ncols()>=CL1 and vertices1[1].ncols()<=CL2:
            cc=[]
            for i in range(len(MatrixGList)):
                cc.append([ MatrixGList[i], vertices1[2][i] ])
            
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            #print('p1,p2',p1,p2)
            
            if p2==None:
                ii1=ContentOfTableauWithMultiplicities(ii)
                p3=repr(ii1)
                p4=b_noncrossing.get(p3)
                if p4==None:
                    u0=NonCrossingTupleWithSameContentAsT_new(ii)
                    print('length of noncrossing tuples', len(u0))
                    b_noncrossing[p3]=u0
                    #print(ii, u0)
                    F2.write(str([ii1, u0])+'\n')
                    
                    
                else:
                    print('b_noncrossing already occur')
                    u0=b_noncrossing[p3]
                
                print('b_noncrossing', len(b_noncrossing))
                
                u1=ChooseTableauxWithSmallOrEqualWeight(ii,u0,typ,rank)
                print('length of noncrossing tuples for ii', len(u1))
                
                
                print('cc', len(cc))
                if len(cc)<=len(u1)+3:
                    t2=[u1, cc] 
                    b6[p1]=t2
                elif len(cc)>len(u1)+3:
                    t3=ComputeChTInTermsOfNonCrossingBasis([u1,cc])
                    print('length of variables and length of solutions', len(u1), len(t3))
                    #print(ii,t3)
                    ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,[[ii,t3]],n)
                    ee1=PluckerPolynomialTableauExpansionsList(ee1,n)
                    ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                    ee1=PromotionOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                  
                    print('ee1', len(ee1))
                    for i in ee1:
                        q1=repr(i[0])
                        q2=b6.get(q1)
                        if q2!=[]:
                            F1.write(str(i)+'\n')
                            b6[q1]=[]  
                            print('number of cluster variables obtained', len(b6))
                    
            elif len(p2)==2 and len(p2[1])<=len(p2[0])+3:
                t1=removeDuplicatesListOfLists(p2[1]+cc)
                print('len(t1)',len(t1),ii)
                if len(t1) != len(p2[1]) + len(cc):  # avoid singular matrix when solving the linear equation to get coefficients
                    b6[p1]=[p2[0], cc]
                else:
                    if len(t1)<=len(p2[0])+3:
                        b6[p1]=[p2[0], t1]
                    else:
                        t2=ComputeChTInTermsOfNonCrossingBasis([p2[0],t1])
                        ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,[[ii,t2]],n)                        
                        ee1=PluckerPolynomialTableauExpansionsList(ee1,n)
                        ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                        ee1=PromotionOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                        print('ee1', len(ee1))
                        for i in ee1:
                            q1=repr(i[0])
                            q2=b6.get(q1)
                            if q2!=[]:
                                F1.write(str(i)+'\n')
                                b6[q1]=[]  
                                print('number of cluster variables obtained', len(b6))
                                
                            
                            
                        
    F1.close()
    F2.close()
                
 
            

#     b1=removeDuplicates2(b1)
#     b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
#     b1=TableauExpansionsInMatrixList(b1, n)
    
     

    return [b6,b_noncrossing]

def initial_cluster_variables_DFGK(k,n): # Grkn
    t1=InitialClusterWithoutNumericalMatrices(k,n)[1][0]
    t2=TakeIthElements(2,t1)
    m=n-k
    r1=[]
    for i in [0..m-2]:
        r2=[]
        for j in [0..k-2]:
            r2.append(t2[j*m+i])
            #print(t2[j*m+i])
        r1=r1+r2
    for i in [m]+[0..m-1]:
            r2=[]
            for j in [k-1]:
                r2.append(t2[j*m+i])
                #print(t2[j*m+i])
            r1=r1+r2
    for i in [m-1]:
            r2=[]
            for j in [0..k-2]:
                r2.append(t2[j*m+i])
                #print(t2[j*m+i])
            r1=r1+r2
    r=r1
    return r



#Function for multiprocessing 
def ll_perms_compute_nonreal_from_clusters_nonreal_only(lli,typ,rank,max_column,n,repeat,b_nonreal,fp2,mutation_number): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ma0=MatrixTakeRows(mat1)
            if len(SetDifference(flatten(ma0), [1,0,-1]))>=2:
                t0=sorted(TakeIthElements(2,vertices1[0]))
                t1=removeAnElementInListList(0,vertices1[0])
                #print('t1',t1)
                t2=[t0, ma0, t1]
                g00=FindNonRealModuleUsingQuiverWithMultipleArrows(rank,n,t2,e1,mutation_number)
                #print(g00)
                if g00!=[]:
                    g0=g00[0][1]
                    if len(g0)<=max_column: # collect only nonreal modules less or equal to max_column columns
                        g1=repr(g0)
                        g2=b_nonreal.get(g1)

                        if g2==None:
                            
                            c1=ExtendSetOfTableauxToContainPromotionsInMatrix([TableauToMatrix(g0)],n) 
                            c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                            c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                            c1=ExtendSetOfTableauxToContainPromotionsInMatrix(c1,n) 
                            c1=MatrixTakeColumnsList(c1)
                            f1=open(fp2, 'a+')
                            for ci in c1:
                                d1=repr(ci)
                                d2=b_nonreal.get(d1)
                                if d2==None:
                                    f1.write(str(ci)+'\n')
                                    b_nonreal[d1]=[] 
                            f1.close()
    return b_nonreal


def has_elements_same_next_to_each_other(l):
    r=0
    for i in range(len(l)-1):
        if l[i]==l[i+1]:
            r=1
            return r
    return r

#Function for multiprocessing 
def ll_perms_compute_clusters_fix_steps(lli,typ,rank,n,b6,bcluster,num_steps,fp1,fp2): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    if has_elements_same_next_to_each_other(ll)==0:
        mutationSequence=ll

        for j in range(len(mutationSequence)): 
            #print('kk')
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 

            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                f1.close()
                b6[p1]=[]

            t0=sorted(TakeIthElements(2,vertices1[0]))
            g1=repr(t0)
            g2=bcluster.get(g1)

            if g2==None:
                f1=open(fp2, 'a+')
                f1.write(str(t0)+'\n')
                f1.close()
                bcluster[g1]=[]


    return [b6, bcluster]


#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix([ii0],n) 
                c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix(c1,n) 
                c1=MatrixTakeColumnsList(c1)
                f1=open(fp1, 'a+')
                for ci in c1:
                    g1=repr(ci)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(ci)+'\n')
                        b6[g1]=[] 
                f1.close()
    return b6


In [2]:
####################################################################
# compute only cluster variables, Grassmannian
# each row in the result produced by these codes is a tableau, for example [[1,2,4],[3,5,6]]. Here 124, 356 are the two columns of the tableau.
if __name__ == '__main__':
    #Define generation hyperparams
    rank, n = 3, 9   #...for Gr(rank, n)
    max_column = 4 #...obtain only tableaux with number of columns less or equal to max_column
    max_step = 6     #...this number controls the length of random mutation sequence, in order to obtain all cluster variables with number of columns less or equal to a fixed number, we need to put the number max_step sufficiently large
    checkpoint = 2   #...if after check_point steps, the number elements in b2 is not increasing, then stop
    repeat=8
    
    fp1='NumericalSmallRankModulesGr'+str(rank)+str(n)+'R'+str(max_column)+'ClusterVariables.txt'
    #fp2='NumericalSmallRankModulesGr'+str(rank)+str(n)+'R'+str(max_column)+'Nonreal.txt'
    
    F2 = open(fp1,'a+')
    F2.close()
    
    #Run generation 
    
    typ=1
    k=rank
    sizeColumn=n-k
    ll0=[] 
    for i in range(1,k):
        for j in range(1,n-k):
            ll0.append((i-1)*sizeColumn+j)
            
    num,num1,sn,sn1=0,0,0,0
    
    b6={}
    fp=open(fp1, 'r')
    l=fp.readline()
    while l :
        j=sage_eval(l)
        #print(j)
        p1=repr(j)
        b6[p1]=[]  
        l=fp.readline()
    fp.close()
    
    print('len(b6)', len(b6)) 
    
    num=0
    num1=0
    num2=0
    while (1):  #...have split the ComputeClusterVariablesInGrkn function up into part we wish to parallelise 'll_perms' and the remainder
        sn=sn+1
        
        #Generate a list of permutations, then run above generation function with them on different cores
        lls = [np.random.permutation(ll0) for iii in range(max_step)]
        
        #print('minn,maxn',minn,maxn)
        
        #F1 = open(fp1,'a+') 
        N_CPU = multiprocessing.cpu_count()
        p = get_context("fork").Pool(processes=N_CPU) # works on mac
        bb = p.starmap(ll_perms_compute_cluster_variables_only,[(lls[i], typ, rank, max_column, n, repeat,b6,fp1) for i in range(len(lls))])
        
        for i in bb:
            #b_nonreal=merge_two_dicts_update(b_nonreal, i)
            b6=merge_two_dicts(b6, i)
             
        #print(len(b_nonreal))
        
        #print('sn',sn)

        #Break loop when all probably generated
        if sn%checkpoint==1:
            print('sn, len(b6)', sn, len(b6))
            num=len(b6)
            if num==num2 and num!=0:
                break
            else:
                num2=num


len(b6) 756
sn, len(b6) 1 756
sn, len(b6) 3 756
